In [46]:
# Import library
import pandas as pd
import numpy as np
import re
import glob
import os
import csv
import json
import psycopg2
import time
from pymongo import MongoClient
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from datetime import datetime as dt
from sqlalchemy import create_engine

# Import specific information
import File_Info as info
import credential as cd


In [2]:
''' Function Collection '''

# Collect files based on nests
def file_based_on_nest_RDR6_TxCal(file_path, golden_flag):
    
    # Get the list of all the file names
    files = os.listdir(file_path)
    
    # Init dataframe and filter files based on the nest number in file names
    Nest_1 = []
    Nest_2 = []
    Nest_3 = []
    Nest_4 = []
    Nest_5 = []
    Nest_6 = []
    Test_result = []
    for file in files:
        if file.startswith("TestOutput_Nest1"):
            Nest_1.append(file_path + file)
        elif file.startswith("TestOutput_Nest2"):
            Nest_2.append(file_path + file)
        elif file.startswith("TestOutput_Nest3"):
            Nest_3.append(file_path + file)
        elif file.startswith("TestOutput_Nest4"):
            Nest_4.append(file_path + file)
        elif file.startswith("TestOutput_Nest5"):
            Nest_5.append(file_path + file)
        elif file.startswith("TestOutput_Nest6"):
            Nest_6.append(file_path + file)
        else:
            Test_result.append(file_path + file)

    if golden_flag == 1:
        return Nest_1, Nest_2, Nest_3, Nest_4, Nest_5, Nest_6
    else:
        return Nest_1, Nest_2, Nest_3, Nest_4, Nest_5, Nest_6, Test_result

    
# Clean column names (remove extra space)
def clean_col_names(names):
    column_names_list = names.split(",")
    clean_names = []
    for i in column_names_list:
        clean_names.append(i.strip())
    return clean_names    


# Concatinate all dataframe based on nests
def combine_csv_RDR6_Txcal(file_path, column_names, total_test_item_num, Golden_FID):
    
    # Clean the column names (remove extra space)
    col_name_clean = clean_col_names(column_names)

    # Initialize an empty dataframe for all files concatenation
    column_name_temp = list(range(100))
    total_data = pd.DataFrame(columns = column_name_temp)

    # Concatenate all the csv files in the folder into one dataframe based on nest
    for item in file_path:
        csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
        total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = True)
    
    # Eliminate redundant/skewed data
    total_data = total_data.iloc[:, :total_test_item_num]
    
    # Name columns
    total_data.columns = col_name_clean
    
    # Filter out test step name
    condition = total_data["Time"] != "Time"
    total_data = total_data[condition]
    
    # Filter out non-golden test records
    if Golden_FID == "0000":
        clean_data = total_data
    else:
        clean_data = total_data[total_data["FID"] == Golden_FID]
        
    # Drop duplicated columns
    clean_data = clean_data.loc[:,~clean_data.columns.duplicated()]

    # Convert data type from string to float for plotting
    try:
        clean_data["Antenna1 GetTxPower"] = clean_data["Antenna1 GetTxPower"].astype("float")
    except ValueError:
        for item in clean_data["Antenna1 GetTxPower"]:
            try:
                item = float(item)
            except ValueError:
                drop_index_1 = clean_data[clean_data["Antenna1 GetTxPower"] == item].index
                clean_data = clean_data.drop(drop_index_1, axis = 0)
                
    try:    
        clean_data["Temperature"] = clean_data["Temperature"].astype("float")
    except ValueError:
        for item in clean_data["Temperature"]:
            try:
                item = float(item)
            except ValueError:
                drop_index_2 = clean_data[clean_data["Temperature"] == item].index
                clean_data = clean_data.drop(drop_index_2, axis = 0)
    
    # Filter out null value
    not_null_condition = clean_data["Antenna1 GetTxPower"].notnull()
    clean_data = clean_data[not_null_condition]

    # Reset index column
    clean_data = clean_data.reset_index(drop = True)
    
    # Convert other data type
    for serial_num in clean_data["MIS"]:
        if len(serial_num) != 12:
            drop_index_3 = clean_data[clean_data["MIS"] == serial_num].index
            clean_data = clean_data.drop(drop_index_3, axis = 0)
    
    clean_data["MIS"] = clean_data["MIS"].astype(np.int64)
    
    # Add Date_time column and set as index column
    clean_data["Date_Time"] = clean_data["TestDate"] + " " + clean_data["Time"]
    clean_data = clean_data.set_index(["Date_Time"])
    
    return clean_data


 # Collect files based on nests for rdr7
def file_based_on_nest_RDR7_Txcal(file_path, golden_flag):
    
    # Get the list of all the file names
    files = os.listdir(file_path)
    
    # Init dataframe and filter files based on the nest number in file names
    Nest_1 = []
    Nest_2 = []
    Nest_3 = []
    Nest_4 = []
    Test_result = []
    for file in files:
        if file.startswith("TestOutput_Nest1"):
            Nest_1.append(file_path + file)
        elif file.startswith("TestOutput_Nest2"):
            Nest_2.append(file_path + file)
        elif file.startswith("TestOutput_Nest3"):
            Nest_3.append(file_path + file)
        elif file.startswith("TestOutput_Nest4"):
            Nest_4.append(file_path + file)
        elif file.startswith("test results"):
            Test_result.append(file_path + file)

    if golden_flag == 1:
        return Nest_1, Nest_2, Nest_3, Nest_4
    else:
        return Nest_1, Nest_2, Nest_3, Nest_4, Test_result

    

# Concatinate all dataframe based on nests for rdr7
def combine_csv_RDR7_Txcal(file_path, column_names, total_test_item_num, Golden_FID, drop_index, golden_flag, prod_type):
    
    # Clean the column names (remove extra space)
    col_name_clean = clean_col_names(column_names)

    # Initialize an empty dataframe for all files concatenation
    column_name_temp = list(range(100))
    total_data = pd.DataFrame()

    # Concatenate all the csv files in the folder into one dataframe based on nest
    if golden_flag == 1:
        for item in file_path:
            csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
            csv_file_sample = csv_file_sample.drop(drop_index, axis = 1)
            total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = True)
    else:
        for item in file_path:
            if prod_type == "1.0":
                old_type = 0
                if datetime.strptime(item[-12:-4], "%Y%m%d") < datetime.strptime("20190805", "%Y%m%d"):
                    old_type = 1
                try:
                    temp_sample = pd.read_csv(item, header = None)
                except pd.errors.ParserError:
                    max_col = 0
                    file_freq_flag = 0
                    for row in open(item):
                        if "b1" in row:
                            file_freq_flag = 1
                            break
                        col_num = len(row.split(","))
                        if col_num > max_col:
                            max_col = col_num
                    if file_freq_flag:
                        temp_sample = pd.read_csv(item, header = None, names = list(range(59)))
                    else:
                        if max_col < 55:
                            temp_sample = pd.read_csv(item, header = None, names = list(range(49)))
                        else:
                            print(f'This file: {item} has unknown contents')
                if len(temp_sample.columns.tolist()) > 55:
                    drop_index_final = [15, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 57]
                else:
                    if old_type:
                        drop_index_final = drop_index + [38, 40]
                    else:
                        drop_index_final = drop_index
            else:
                drop_index_final = drop_index
            csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
            csv_file_sample = csv_file_sample.drop(drop_index_final, axis = 1)
            csv_file_sample = csv_file_sample.iloc[:, :total_test_item_num]
            csv_file_sample.columns = list(range(0, total_test_item_num))
            total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = False)

    # Eliminate redundant/skewed data
    total_data = total_data.iloc[:, :total_test_item_num]

    # Name columns
    total_data.columns = col_name_clean

    # Filter out test step name
    condition = total_data["Time"] != "Time"
    total_data = total_data[condition]

    # Filter out non-golden test records
    if Golden_FID == "0000":
        clean_data = total_data
    else:
        clean_data = total_data[total_data["FID"] == Golden_FID]

    # Drop duplicated columns
    clean_data = clean_data.loc[:,~clean_data.columns.duplicated()]

    clean_data.to_csv("just-check.csv")
    # Convert data type from string to float
    try:
        clean_data["Antenna1 GetTxPower"] = clean_data["Antenna1 GetTxPower"].astype("float")
    except ValueError:
        for item in clean_data["Antenna1 GetTxPower"]:
            try:
                item = float(item)
            except ValueError:
                drop_index_1 = clean_data[clean_data["Antenna1 GetTxPower"] == item].index
                clean_data = clean_data.drop(drop_index_1, axis = 0)

    try:    
        clean_data["Temperature"] = clean_data["Temperature"].astype("float")
    except ValueError:
        for item in clean_data["Temperature"]:
            try:
                item = float(item)
            except ValueError:
                drop_index_2 = clean_data[clean_data["Temperature"] == item].index
                clean_data = clean_data.drop(drop_index_2, axis = 0)

    # Filter out null value
    not_null_condition = clean_data["Antenna1 GetTxPower"].notnull()
    clean_data = clean_data[not_null_condition]

    # Reset index column
    clean_data = clean_data.reset_index(drop = True)

    clean_data = clean_data[~clean_data["MIS"].isnull()]
    try:
        clean_data["MIS"] = clean_data["MIS"].astype(np.int64)
    except ValueError:
        clean_data["MIS"] = clean_data["MIS"].apply(lambda x: int(float(x)))
        
    clean_data["MIS"] = clean_data["MIS"].astype(str)

    # Convert other data type
    for serial_num in clean_data["MIS"]:
        if len(serial_num) != 12:
            drop_index_3 = clean_data[clean_data["MIS"] == serial_num].index
            clean_data = clean_data.drop(drop_index_3, axis = 0)

    # Add Date_time column and set as index column
    clean_data["Date_Time"] = clean_data["Date"] + " " + clean_data["Time"]
    clean_data = clean_data.set_index(["Date_Time"])
    
    return clean_data
    

# Process test result files
def combine_test_result_Txcal(file_path, column_names, total_test_item_num):
    
    # Clean the column names (remove extra space)
    column_names_list = column_names.split(",")
    col_name_clean = []
    for i in column_names_list:
        col_name_clean.append(i.strip())
    
    # Initialize an empty dataframe for all files concatenation
    column_name_temp = list(range(200))
    total_data = pd.DataFrame(columns = column_name_temp)

    # Concatenate all the csv files in the folder into one dataframe based on nest
    for item in file_path:
        csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
        total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = True)
    
    # Eliminate redundant/skewed data
    total_data = total_data.iloc[:, :total_test_item_num]
    
    # Name columns
    total_data.columns = col_name_clean
    
    # Filter out test step name
    condition = total_data["Time"] != "Time"
    total_data = total_data[condition]
    
    # Eliminate NaN row
    is_null = total_data["MIS"].isnull()
    total_data = total_data[~is_null].reset_index(drop = True)
    
    return total_data

# Merge the test result with production log files
def merge_prod_test(df):
    df["MIS"] = df["MIS"].astype(str)
    df["Nest"] = df["Nest"].astype(str)
    try:
        df["Date"] = df["Date"].astype(str)
        df["identity"] = df["Date"] + "_" + df["MIS"] + "_" + df["Nest"]
    except KeyError:
        df["TestDate"] = df["TestDate"].astype(str)
        df["identity"] = df["TestDate"] + "_" + df["MIS"] + "_" + df["Nest"]
    df = df.set_index(["identity"])
    return df

# RDR6

## TxCal

### Golden

#### GM175 Product

In [9]:
# Locate all the files
RDR6_TxCal_Golden_GM175_file_path = "../Final Project - Log Collection/RDR6/TxCal/Golden/GM175/GM175_GOLD/"

gm175_golden_nest1, gm175_golden_nest2, gm175_golden_nest3, gm175_golden_nest4, gm175_golden_nest5, gm175_golden_nest6 = file_based_on_nest_RDR6_Txcal(RDR6_TxCal_Golden_GM175_file_path, 1)

In [10]:
# Process all the files
RDR6_Txcal_Golden_GM175_nest_1_df = combine_csv_RDR6_Txcal(gm175_golden_nest1, info.column_names_GM175_golden, info.total_test_item_golden_RDR6_GM175, info.RDR6_TxCal_golden_GM175_FID)
RDR6_Txcal_Golden_GM175_nest_2_df = combine_csv_RDR6_Txcal(gm175_golden_nest2, info.column_names_GM175_golden, info.total_test_item_golden_RDR6_GM175, info.RDR6_TxCal_golden_GM175_FID)
RDR6_Txcal_Golden_GM175_nest_3_df = combine_csv_RDR6_Txcal(gm175_golden_nest3, info.column_names_GM175_golden, info.total_test_item_golden_RDR6_GM175, info.RDR6_TxCal_golden_GM175_FID)
RDR6_Txcal_Golden_GM175_nest_4_df = combine_csv_RDR6_Txcal(gm175_golden_nest4, info.column_names_GM175_golden, info.total_test_item_golden_RDR6_GM175, info.RDR6_TxCal_golden_GM175_FID)
RDR6_Txcal_Golden_GM175_nest_5_df = combine_csv_RDR6_Txcal(gm175_golden_nest5, info.column_names_GM175_golden, info.total_test_item_golden_RDR6_GM175, info.RDR6_TxCal_golden_GM175_FID)
RDR6_Txcal_Golden_GM175_nest_6_df = combine_csv_RDR6_Txcal(gm175_golden_nest6, info.column_names_GM175_golden, info.total_test_item_golden_RDR6_GM175, info.RDR6_TxCal_golden_GM175_FID)

<ipython-input-5-36ebaf6b9a0e>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Antenna1 GetTxPower"] = clean_data["Antenna1 GetTxPower"].astype("float")
<ipython-input-5-36ebaf6b9a0e>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Temperature"] = clean_data["Temperature"].astype("float")


In [9]:
# Add Nest column to each dataframe
RDR6_Txcal_Golden_GM175_nest_1_df["Nest"] = 1
RDR6_Txcal_Golden_GM175_nest_2_df["Nest"] = 2
RDR6_Txcal_Golden_GM175_nest_3_df["Nest"] = 3
RDR6_Txcal_Golden_GM175_nest_4_df["Nest"] = 4
RDR6_Txcal_Golden_GM175_nest_5_df["Nest"] = 5
RDR6_Txcal_Golden_GM175_nest_6_df["Nest"] = 6

In [10]:
# Form final dataframe
RDR6_TxCal_Golden_GM175 = pd.concat(objs = [RDR6_Txcal_Golden_GM175_nest_1_df, RDR6_Txcal_Golden_GM175_nest_2_df, RDR6_Txcal_Golden_GM175_nest_3_df, RDR6_Txcal_Golden_GM175_nest_4_df, RDR6_Txcal_Golden_GM175_nest_5_df, RDR6_Txcal_Golden_GM175_nest_6_df], ignore_index = False)

In [ ]:
# Set column name properly for SQL loading
RDR6_TxCal_Golden_GM175.columns = info.column_names_GM175_golden_SQL

In [11]:
# Output as csv
RDR6_TxCal_Golden_GM175.to_csv("../Data_clean/RDR6_TxCal_Golden_GM175.csv")

#### Chrysler Product

In [32]:
RDR6_TxCal_Golden_Chrysler_file_path = "../Final Project - Log Collection/RDR6/TxCal/Golden/Chrysler/Chrysler/"
chrysler_golden_nest1, chrysler_golden_nest2, chrysler_golden_nest3, chrysler_golden_nest4, chrysler_golden_nest5, chrysler_golden_nest6 = file_based_on_nest_RDR6_Txcal(RDR6_TxCal_Golden_Chrysler_file_path, 1)

In [33]:
# Process all the files
RDR6_Txcal_Golden_Chrysler_nest_1_df = combine_csv_RDR6_Txcal(chrysler_golden_nest1, info.column_names_Chrysler_golden, info.total_test_item_golden_RDR6_Chrysler, info.RDR6_TxCal_golden_Chrysler_FID)
RDR6_Txcal_Golden_Chrysler_nest_2_df = combine_csv_RDR6_Txcal(chrysler_golden_nest2, info.column_names_Chrysler_golden, info.total_test_item_golden_RDR6_Chrysler, info.RDR6_TxCal_golden_Chrysler_FID)
RDR6_Txcal_Golden_Chrysler_nest_3_df = combine_csv_RDR6_Txcal(chrysler_golden_nest3, info.column_names_Chrysler_golden, info.total_test_item_golden_RDR6_Chrysler, info.RDR6_TxCal_golden_Chrysler_FID)
RDR6_Txcal_Golden_Chrysler_nest_4_df = combine_csv_RDR6_Txcal(chrysler_golden_nest4, info.column_names_Chrysler_golden, info.total_test_item_golden_RDR6_Chrysler, info.RDR6_TxCal_golden_Chrysler_FID)
RDR6_Txcal_Golden_Chrysler_nest_5_df = combine_csv_RDR6_Txcal(chrysler_golden_nest5, info.column_names_Chrysler_golden, info.total_test_item_golden_RDR6_Chrysler, info.RDR6_TxCal_golden_Chrysler_FID)
RDR6_Txcal_Golden_Chrysler_nest_6_df = combine_csv_RDR6_Txcal(chrysler_golden_nest6, info.column_names_Chrysler_golden, info.total_test_item_golden_RDR6_Chrysler, info.RDR6_TxCal_golden_Chrysler_FID)


In [34]:
# Add Nest column to each dataframe
RDR6_Txcal_Golden_Chrysler_nest_1_df["Nest"] = 1
RDR6_Txcal_Golden_Chrysler_nest_2_df["Nest"] = 2
RDR6_Txcal_Golden_Chrysler_nest_3_df["Nest"] = 3
RDR6_Txcal_Golden_Chrysler_nest_4_df["Nest"] = 4
RDR6_Txcal_Golden_Chrysler_nest_5_df["Nest"] = 5
RDR6_Txcal_Golden_Chrysler_nest_6_df["Nest"] = 6

In [42]:
# Form final dataframe
RDR6_TxCal_Golden_Chrysler = pd.concat(objs = [RDR6_Txcal_Golden_Chrysler_nest_1_df, RDR6_Txcal_Golden_Chrysler_nest_2_df, RDR6_Txcal_Golden_Chrysler_nest_3_df, RDR6_Txcal_Golden_Chrysler_nest_4_df, RDR6_Txcal_Golden_Chrysler_nest_5_df, RDR6_Txcal_Golden_Chrysler_nest_6_df], ignore_index = False)

In [ ]:
# Set column name properly for SQL loading
RDR6_TxCal_Golden_Chrysler.columns = info.column_names_Chrysler_golden_SQL

In [60]:
# Output as csv
RDR6_TxCal_Golden_Chrysler.to_csv("../Data_clean/RDR6_TxCal_Golden_Chrysler.csv")

### Production

#### GM175 Product

In [41]:
# Locate all the files
RDR6_TxCal_Prod_GM175_file_path = "../Final Project - Log Collection/RDR6/TxCal/Production/GM175/GM175 production log/"

gm175_prod_nest1, gm175_prod_nest2, gm175_prod_nest3, gm175_prod_nest4, gm175_prod_nest5, gm175_prod_nest6, gm175_prod_test_result = file_based_on_nest_RDR6_Txcal(RDR6_TxCal_Prod_GM175_file_path, 0)

In [47]:
# Process all the files
RDR6_Txcal_Prod_GM175_nest_1_df = combine_csv_RDR6_Txcal(gm175_prod_nest1, info.column_names_GM175_prod, info.total_test_item_prod_RDR6_GM175, info.RDR6_TxCal_prod_GM175_FID)
RDR6_Txcal_Prod_GM175_nest_2_df = combine_csv_RDR6_Txcal(gm175_prod_nest2, info.column_names_GM175_prod, info.total_test_item_prod_RDR6_GM175, info.RDR6_TxCal_prod_GM175_FID)
RDR6_Txcal_Prod_GM175_nest_3_df = combine_csv_RDR6_Txcal(gm175_prod_nest3, info.column_names_GM175_prod, info.total_test_item_prod_RDR6_GM175, info.RDR6_TxCal_prod_GM175_FID)
RDR6_Txcal_Prod_GM175_nest_4_df = combine_csv_RDR6_Txcal(gm175_prod_nest4, info.column_names_GM175_prod, info.total_test_item_prod_RDR6_GM175, info.RDR6_TxCal_prod_GM175_FID)
RDR6_Txcal_Prod_GM175_nest_5_df = combine_csv_RDR6_Txcal(gm175_prod_nest5, info.column_names_GM175_prod, info.total_test_item_prod_RDR6_GM175, info.RDR6_TxCal_prod_GM175_FID)
RDR6_Txcal_Prod_GM175_nest_6_df = combine_csv_RDR6_Txcal(gm175_prod_nest6, info.column_names_GM175_prod, info.total_test_item_prod_RDR6_GM175, info.RDR6_TxCal_prod_GM175_FID)

In [42]:
# Combine Test result files
gm175_prod_test_result_df = combine_test_result_Txcal(gm175_prod_test_result, info.RDR6_test_result_col_names, info.RDR6_total_test_result_item_num)

In [48]:
# Add Nest column to each dataframe
RDR6_Txcal_Prod_GM175_nest_1_df["Nest"] = 1
RDR6_Txcal_Prod_GM175_nest_2_df["Nest"] = 2
RDR6_Txcal_Prod_GM175_nest_3_df["Nest"] = 3
RDR6_Txcal_Prod_GM175_nest_4_df["Nest"] = 4
RDR6_Txcal_Prod_GM175_nest_5_df["Nest"] = 5
RDR6_Txcal_Prod_GM175_nest_6_df["Nest"] = 6

In [49]:
# Form final dataframe
RDR6_TxCal_Prod_GM175 = pd.concat(objs = [RDR6_Txcal_Prod_GM175_nest_1_df, RDR6_Txcal_Prod_GM175_nest_2_df, RDR6_Txcal_Prod_GM175_nest_3_df, RDR6_Txcal_Prod_GM175_nest_4_df, RDR6_Txcal_Prod_GM175_nest_5_df, RDR6_Txcal_Prod_GM175_nest_6_df], ignore_index = False)

In [45]:
# Output Log as csv
RDR6_TxCal_Prod_GM175.to_csv("../Data_clean/RDR6_TxCal_Prod_GM175.csv")

In [ ]:
# Output test result as csv
gm175_prod_test_result_df.to_csv("../Data_clean/RDR6_TxCal_Prod_GM175_Test_Result.csv", index = False)

#### Chrysler Product

In [47]:
RDR6_TxCal_Prod_Chrysler_file_path = "../Final Project - Log Collection/RDR6/TxCal/Production/Chrysler/Chrysler/"
chrysler_prod_nest1, chrysler_prod_nest2, chrysler_prod_nest3, chrysler_prod_nest4, chrysler_prod_nest5, chrysler_prod_nest6, chrysler_prod_test_result = file_based_on_nest_RDR6_Txcal(RDR6_TxCal_Prod_Chrysler_file_path, 0)

In [52]:
# Process all the files
RDR6_Txcal_Prod_Chrysler_nest_1_df = combine_csv_RDR6_Txcal(chrysler_prod_nest1, info.column_names_Chrysler_prod, info.total_test_item_prod_RDR6_Chrysler, info.RDR6_TxCal_prod_Chrysler_FID)
RDR6_Txcal_Prod_Chrysler_nest_2_df = combine_csv_RDR6_Txcal(chrysler_prod_nest2, info.column_names_Chrysler_prod, info.total_test_item_prod_RDR6_Chrysler, info.RDR6_TxCal_prod_Chrysler_FID)
RDR6_Txcal_Prod_Chrysler_nest_3_df = combine_csv_RDR6_Txcal(chrysler_prod_nest3, info.column_names_Chrysler_prod, info.total_test_item_prod_RDR6_Chrysler, info.RDR6_TxCal_prod_Chrysler_FID)
RDR6_Txcal_Prod_Chrysler_nest_4_df = combine_csv_RDR6_Txcal(chrysler_prod_nest4, info.column_names_Chrysler_prod, info.total_test_item_prod_RDR6_Chrysler, info.RDR6_TxCal_prod_Chrysler_FID)
RDR6_Txcal_Prod_Chrysler_nest_5_df = combine_csv_RDR6_Txcal(chrysler_prod_nest5, info.column_names_Chrysler_prod, info.total_test_item_prod_RDR6_Chrysler, info.RDR6_TxCal_prod_Chrysler_FID)
RDR6_Txcal_Prod_Chrysler_nest_6_df = combine_csv_RDR6_Txcal(chrysler_prod_nest6, info.column_names_Chrysler_prod, info.total_test_item_prod_RDR6_Chrysler, info.RDR6_TxCal_prod_Chrysler_FID)


In [48]:
# Combine Test result files
chrysler_prod_test_result_df = combine_test_result_Txcal(chrysler_prod_test_result, info.RDR6_test_result_col_names, info.RDR6_total_test_result_item_num)

D:\Anaconda\software\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [53]:
# Add Nest column to each dataframe
RDR6_Txcal_Prod_Chrysler_nest_1_df["Nest"] = 1
RDR6_Txcal_Prod_Chrysler_nest_2_df["Nest"] = 2
RDR6_Txcal_Prod_Chrysler_nest_3_df["Nest"] = 3
RDR6_Txcal_Prod_Chrysler_nest_4_df["Nest"] = 4
RDR6_Txcal_Prod_Chrysler_nest_5_df["Nest"] = 5
RDR6_Txcal_Prod_Chrysler_nest_6_df["Nest"] = 6

In [54]:
# Form final dataframe
RDR6_TxCal_Prod_Chrysler = pd.concat(objs = [RDR6_Txcal_Prod_Chrysler_nest_1_df, RDR6_Txcal_Prod_Chrysler_nest_2_df, RDR6_Txcal_Prod_Chrysler_nest_3_df, RDR6_Txcal_Prod_Chrysler_nest_4_df, RDR6_Txcal_Prod_Chrysler_nest_5_df, RDR6_Txcal_Prod_Chrysler_nest_6_df], ignore_index = False)

In [55]:
# Output as csv
RDR6_TxCal_Prod_Chrysler.to_csv("../Data_clean/RDR6_TxCal_Prod_Chrysler.csv")

In [51]:
# Output test result as csv
chrysler_prod_test_result_df.to_csv("../Data_clean/RDR6_TxCal_Prod_Chrysler_Test_Result.csv", index = False)

#### Combine to final log file

In [64]:
# For GM175
# Prod
gm175_df = pd.read_csv("../Data_clean/RDR6_TxCal_Prod_GM175.csv", parse_dates = ["TestDate"])
gm175_df = merge_prod_test(gm175_df)

# test result
gm175_test = pd.read_csv("../Data_clean/RDR6_TxCal_Prod_GM175_Test_Result.csv", parse_dates = ["TestDate"])
gm175_test = merge_prod_test(gm175_test)

# merge
gm175_temp = gm175_df.merge(gm175_test, how = "left", left_index = True, right_index = True)
gm175 = gm175_temp[~gm175_temp["Pass/Fail"].isnull()]
gm175 = gm175[["Date_Time", "TestDate", "Nest_x", "Time_x", "Test Time", "Pass/Fail", "Current", "MIS_x", "FID", "Temperature", "RCTA Occupied BW", "Tx1 BSD", "Tx1 RCTA", "Tx2 BSD", "Tx2 RCTA", "Antenna1 GetTxPower", "Antenna2 GetTxPower", "Antenna1 GetCenterFrequency", "Antenna2 GetCenterFrequency"]]
gm175_final = gm175.rename(columns = {'Nest_x' : "Nest", "Time_x": "Time", "MIS_x" : "MIS"})

D:\Anaconda\software\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12,17,21,24,51,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date_Time,TestDate,Time,Bat_Ctrl,Current,InitCanInterface,Sensor_Comm_Init,Get_Test_Instrument,Init_Tx_Cal,PingDut,...,Antenna2 GetTxPower,Antenna1 GetCenterFrequency,Antenna2 GetCenterFrequency,Antenna1 GetCwLeakage,Antenna2 GetCwLeakage,Delta,SerialpointsCheck,ReadPointCSV,VerifyNBMailboxPoints,Nest
0,2020-01-02 17:48:21,2020-01-02,17:48:21,1,296.668,Connection_Status=0,Sensor_Comm_Init=0,1,InitTxCal=0,PING=1,...,15.467917,24.060036,24.060029,28.425758,28.749436,5.86039,No mailbox file,NaN,0.0,1
1,2020-01-02 17:51:43,2020-01-02,17:51:43,1,295.336,Connection_Status=0,Sensor_Comm_Init=0,1,InitTxCal=0,PING=1,...,15.260054,24.059949,24.059943,29.049969,28.812460,5.64448,No mailbox file,NaN,0.0,1
2,2020-01-02 17:53:03,2020-01-02,17:53:03,1,288.000,Connection_Status=0,Sensor_Comm_Init=0,1,InitTxCal=0,PING=1,...,15.394069,24.059882,24.059876,28.977403,28.434372,5.70617,No mailbox file,NaN,0.0,1
3,2020-01-02 17:54:18,2020-01-02,17:54:18,1,296.982,Connection_Status=0,Sensor_Comm_Init=0,1,InitTxCal=0,PING=1,...,15.526482,24.060068,24.060061,28.429197,29.180168,5.49026,No mailbox file,NaN,0.0,1
4,2020-01-02 18:26:56,2020-01-02,18:26:56,1,295.950,Connection_Status=0,Sensor_Comm_Init=0,1,InitTxCal=0,PING=1,...,15.394393,24.060094,24.060086,28.919926,29.031467,6.48287,No mailbox file,NaN,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315599,2021-05-18 00:04:51,2021-05-18,00:04:51,1,294.514,Connection_Status=0,Sensor_Comm_Init=0,3,InitTxCal=0,PING=1,...,15.464454,24.059327,24.059320,30.330945,31.363809,4.96592,No mailbox file,NaN,0.0,6
315600,2021-05-18 00:06:21,2021-05-18,00:06:21,1,290.733,Connection_Status=0,Sensor_Comm_Init=0,3,InitTxCal=0,PING=1,...,15.391666,24.059252,24.059245,30.029807,30.460086,4.90422,No mailbox file,NaN,0.0,6
315601,2021-05-18 00:08:28,2021-05-18,00:08:28,1,270.139,Connection_Status=0,Sensor_Comm_Init=0,3,InitTxCal=0,PING=1,...,15.404977,24.059404,24.059398,30.984338,31.097889,5.36688,No mailbox file,NaN,0.0,6
315602,2021-05-18 00:09:18,2021-05-18,00:09:18,1,295.527,Connection_Status=0,Sensor_Comm_Init=0,3,InitTxCal=0,PING=1,...,15.457123,24.059389,24.059383,29.654461,29.888180,4.99676,No mailbox file,NaN,0.0,6


In [ ]:
# Output (if needed) - GM175
gm175_final.to_csv("../Data_clean/RDR6_TxCal_Prod_GM175_FINAL.csv", index = False)

In [ ]:
# For Chrysler
# Prod
chrysler_df = pd.read_csv("../Data_clean/RDR6_TxCal_Prod_Chrysler.csv")
chrysler_df = merge_prod_test(chrysler_df)

# test result
chrysler_test = pd.read_csv("../Data_clean/RDR6_TxCal_Prod_Chrysler_Test_Result.csv")
chrysler_test = merge_prod_test(chrysler_test)

# merge
chrysler_temp = chrysler_df.merge(chrysler_test, how = "left", left_index = True, right_index = True)
chrysler = chrysler_temp[~chrysler_temp["Pass/Fail"].isnull()]
chrysler = chrysler[["Date_Time", "TestDate", "Nest_x", "Time_x", "Test Time", "Pass/Fail", "Current", "MIS_x", "Temperature", "RCTA Occupied BW", "BSD Occupied BW", "Power", "Antenna1 GetTxPower", "Antenna2 GetTxPower", "Antenna1 GetCenterFrequency", "Antenna2 GetCenterFrequency"]]
chrysler_final = chrysler.rename(columns = {'Nest_x' : "Nest", "Time_x": "Time", "MIS_x" : "MIS"})


In [161]:
# Output (if needed) - Chrysler
chrysler_final.to_csv("../Data_clean/RDR6_TxCal_Prod_Chrysler_FINAL.csv", index = False)

## Sensor Test

In [12]:
# Collect files based on nests
def file_based_on_nest_RDR6_sensor_test(file_path, golden_flag):
    
    # Get the list of all the file names
    files = os.listdir(file_path)
    
    # Init the file collection name
    gm175_ST = []
    chrysler_ST = []
    
    if golden_flag:
        
        # Init dataframe and filter files based on the nest number in file names
        for file in files:
            if file.startswith("GM175"):
                gm175_ST.append(file_path + file)
            else:
                chrysler_ST.append(file_path + file)
    
    else:
        gm_1_0_ST = []
        failure_ST = []
        
        # Init dataframe and filter files based on the nest number in file names
        for file in files:
            if file.startswith("GM175"):
                gm175_ST.append(file_path + file)
            elif file.startswith("14"):
                gm_1_0_ST.append(file_path + file)
            elif file.startswith("15") or file.startswith("17") or file.startswith("18"):
                chrysler_ST.append(file_path + file)
            else:
                failure_ST.append(file_path + file)
    
    return gm175_ST, chrysler_ST



# Concatinate all dataframe based on nests
def combine_csv_RDR6_sensor_test(file_path, column_names, total_test_item_num, golden_flag, prod_type):
    
    if golden_flag:
        # Clean the column names (remove extra space)
        col_name_clean = clean_col_names(column_names)

        # Initialize an empty dataframe for all files concatenation
        column_name_temp = list(range(200))
        total_data = pd.DataFrame(columns = column_name_temp)

        # Concatenate all the csv files in the folder into one dataframe based on nest
        for item in file_path:
            csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
            total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = True)

        # Eliminate redundant/skewed data
        total_data = total_data.iloc[:, :total_test_item_num]

        # Name columns
        total_data.columns = col_name_clean

        # Filter out test step name
        condition = total_data["Time"] != "Time"
        total_data = total_data[condition]
        clean_data = total_data[~total_data["Time"].isnull()]

        # Drop duplicated columns
        clean_data = clean_data.loc[:,~clean_data.columns.duplicated()]

        # Convert data type from string to float for plotting
        if prod_type == "GM175":

            # Convert to float data type.
            for col_name_temp in ["GM1.75 Angle @ -30", "GM1.75 Angle @ 0", "GM1.75 Angle @ 30"]:
                try:
                    clean_data[col_name_temp] = clean_data[col_name_temp].astype("float")
                except ValueError:
                    for item in clean_data[col_name_temp]:
                        try:
                            item = float(item)
                        except ValueError:
                            print(item)
                            drop_index_tmp = clean_data[clean_data[col_name_temp] == item].index
                            clean_data = clean_data.drop(drop_index_tmp, axis = 0)

        else:
            clean_data["Normalized Tx2 Power @ -30"] = clean_data["Normalized Tx2 Power @ -30"].astype("float")
            clean_data["Normalized Tx2 Power @ 0"] = clean_data["Normalized Tx2 Power @ 0"].astype("float")
            clean_data["Normalized Tx2 Power @ 30"] = clean_data["Normalized Tx2 Power @ 30"].astype("float")
            clean_data["RCTA Main SNR @ 0"] = clean_data["RCTA Main SNR @ 0"].astype("float")
            clean_data["Tx1 EIRP"] = clean_data["Tx1 EIRP"].astype("float")

    else:
        # Initialize an empty dataframe for all files concatenation
        total_data = pd.DataFrame()
        col_final = column_names.split(",")

        # Concatenate all the csv files in the folder into one dataframe based on nest
        for item in file_path:
            file_temp = list()
            max_col = 0
            with open(item, newline = '') as file:
                reader = csv.reader(file, delimiter = ",")
                row_1 = next(reader)
                file_temp.append(row_1)
                max_col = len(row_1)
                if prod_type == "GM175":
                    if max_col < 50:
                        continue
                if prod_type == "Chrysler":
                    if max_col < 70:
                        continue
                row_1[0] = "Date"
                row_1[1] = "Time"
                row_1[2] = "SN"
                row_1[3] = "Nest"
                row_1[4] = "Pass/Fail"

                for row in reader:
                    if len(row) == max_col:
                        file_temp.append(row)
                df = pd.DataFrame(file_temp, columns = row_1)
                df = df.loc[:,~df.columns.duplicated()]
                df = df[col_final]
                total_data = pd.concat(objs = [total_data, df], ignore_index = True)

        # Filter out test step name
        condition = total_data["Time"] != "Time"
        total_data = total_data[condition]
        total_data = total_data[pd.to_numeric(total_data["Nest"], errors = "coerce").notnull()]

        # Convert data type from string to float for plotting
        if prod_type == "GM175":
            total_data = total_data[pd.to_numeric(total_data["GM1.75 Angle @ -30"], errors = "coerce").notnull()]
            total_data = total_data[pd.to_numeric(total_data["GM1.75 Angle @ 0"], errors = "coerce").notnull()]
            clean_data = total_data[pd.to_numeric(total_data["GM1.75 Angle @ 30"], errors = "coerce").notnull()]

            clean_data["GM1.75 Angle @ -30"] = clean_data["GM1.75 Angle @ -30"].astype("float")
            clean_data["GM1.75 Angle @ 0"] = clean_data["GM1.75 Angle @ 0"].astype("float")
            clean_data["GM1.75 Angle @ 30"] = clean_data["GM1.75 Angle @ 30"].astype("float")

        else:
            total_data = total_data[pd.to_numeric(total_data["Normalized Tx2 Power @ -20"], errors = "coerce").notnull()]
            total_data = total_data[pd.to_numeric(total_data["Normalized Tx2 Power @ 0"], errors = "coerce").notnull()]
            total_data = total_data[pd.to_numeric(total_data["Normalized Tx2 Power @ 20"], errors = "coerce").notnull()]
            clean_data = total_data[pd.to_numeric(total_data["Tx1 EIRP"], errors = "coerce").notnull()]

            clean_data["Normalized Tx2 Power @ -30"] = clean_data["Normalized Tx2 Power @ -20"].astype("float")
            clean_data["Normalized Tx2 Power @ 0"] = clean_data["Normalized Tx2 Power @ 0"].astype("float")
            clean_data["Normalized Tx2 Power @ 30"] = clean_data["Normalized Tx2 Power @ 20"].astype("float")
            clean_data["Tx1 EIRP"] = clean_data["Tx1 EIRP"].astype("float")
    
    # Reset index column
    clean_data = clean_data.reset_index(drop = True)

    # Convert other data type
    clean_data["SN"] = clean_data["SN"].fillna("000000")

    # Add Date_time column and set as index column
    clean_data["Date_Time"] = clean_data["Date"] + " " + clean_data["Time"]
    clean_data = clean_data.set_index(["Date_Time"])
    
    return clean_data

### Golden

#### GM175 Product

In [4]:
# Sensor Test golden files directory
RDR6_ST1_Golden_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST1/Golden/Golden/"
RDR6_ST2_Golden_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST2/Golden/Golden/"
RDR6_ST3_Golden_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST3/Golden/Golden/"
RDR6_ST4_Golden_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST4/Golden/Golden/"
RDR6_ST5_Golden_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST5/Golden/Golden/"

In [5]:
# Locate all the golden ST files
gm175_ST1_golden_files, chrysler_ST1_golden_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST1_Golden_file_path, 1)
gm175_ST2_golden_files, chrysler_ST2_golden_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST2_Golden_file_path, 1)
gm175_ST3_golden_files, chrysler_ST3_golden_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST3_Golden_file_path, 1)
gm175_ST4_golden_files, chrysler_ST4_golden_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST4_Golden_file_path, 1)
gm175_ST5_golden_files, chrysler_ST5_golden_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST5_Golden_file_path, 1)

In [3]:
# Process for GM175 ST golden files
gm175_ST1_golden_df = combine_csv_RDR6_sensor_test(gm175_ST1_golden_files, info.ST_column_names_GM175_golden, info.ST_total_test_item_golden_RDR6_GM175, 1, "GM175")
gm175_ST2_golden_df = combine_csv_RDR6_sensor_test(gm175_ST2_golden_files, info.ST_column_names_GM175_golden, info.ST_total_test_item_golden_RDR6_GM175, 1, "GM175")
gm175_ST3_golden_df = combine_csv_RDR6_sensor_test(gm175_ST3_golden_files, info.ST_column_names_GM175_golden, info.ST_total_test_item_golden_RDR6_GM175, 1, "GM175")
gm175_ST4_golden_df = combine_csv_RDR6_sensor_test(gm175_ST4_golden_files, info.ST_column_names_GM175_golden, info.ST_total_test_item_golden_RDR6_GM175, 1, "GM175")
gm175_ST5_golden_df = combine_csv_RDR6_sensor_test(gm175_ST5_golden_files, info.ST_column_names_GM175_golden, info.ST_total_test_item_golden_RDR6_GM175, 1, "GM175")

In [33]:
# Add Nest column to each dataframe
gm175_ST1_golden_df["Tester_num"] = 1
gm175_ST2_golden_df["Tester_num"] = 2
gm175_ST3_golden_df["Tester_num"] = 3
gm175_ST4_golden_df["Tester_num"] = 4
gm175_ST5_golden_df["Tester_num"] = 5

In [34]:
# Form final dataframe
RDR6_ST_Golden_GM175 = pd.concat(objs = [gm175_ST1_golden_df, gm175_ST2_golden_df, gm175_ST3_golden_df, gm175_ST4_golden_df, gm175_ST5_golden_df], ignore_index = False)

In [35]:
# Output Log as csv
RDR6_ST_Golden_GM175.to_csv("../Data_clean/RDR6_ST_Golden_GM175.csv")

#### Chrysler Product

In [6]:
# Process for Chrysler ST golden files
chrysler_ST1_golden_df = combine_csv_RDR6_sensor_test(chrysler_ST1_golden_files, info.ST_column_names_Chrysler_golden, info.ST_total_test_item_golden_RDR6_Chrysler, 1, "Chrysler")
chrysler_ST2_golden_df = combine_csv_RDR6_sensor_test(chrysler_ST2_golden_files, info.ST_column_names_Chrysler_golden, info.ST_total_test_item_golden_RDR6_Chrysler, 1, "Chrysler")
chrysler_ST3_golden_df = combine_csv_RDR6_sensor_test(chrysler_ST3_golden_files, info.ST_column_names_Chrysler_golden, info.ST_total_test_item_golden_RDR6_Chrysler, 1, "Chrysler")
chrysler_ST4_golden_df = combine_csv_RDR6_sensor_test(chrysler_ST4_golden_files, info.ST_column_names_Chrysler_golden, info.ST_total_test_item_golden_RDR6_Chrysler, 1, "Chrysler")
chrysler_ST5_golden_df = combine_csv_RDR6_sensor_test(chrysler_ST5_golden_files, info.ST_column_names_Chrysler_golden, info.ST_total_test_item_golden_RDR6_Chrysler, 1, "Chrysler")

In [8]:
# Add Nest column to each dataframe
chrysler_ST1_golden_df["Tester_num"] = 1
chrysler_ST2_golden_df["Tester_num"] = 2
chrysler_ST3_golden_df["Tester_num"] = 3
chrysler_ST4_golden_df["Tester_num"] = 4
chrysler_ST5_golden_df["Tester_num"] = 5

In [9]:
# Form final dataframe
RDR6_ST_Golden_Chrysler = pd.concat(objs = [chrysler_ST1_golden_df, chrysler_ST2_golden_df, chrysler_ST3_golden_df, chrysler_ST4_golden_df, chrysler_ST5_golden_df], ignore_index = False)

In [10]:
# Output Log as csv
RDR6_ST_Golden_Chrysler.to_csv("../Data_clean/RDR6_ST_Golden_Chrysler.csv")

## Production

### GM 1.75

In [13]:
# Sensor Test production files directory
RDR6_ST1_Prod_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST1/Production/Test Data/"
RDR6_ST2_Prod_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST2/Production/Test Data/"
RDR6_ST3_Prod_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST3/Production/Test Data/"
RDR6_ST4_Prod_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST4/Production/Test Data/"
RDR6_ST5_Prod_file_path = "../Final Project - Log Collection/RDR6/Sensor Test/ST5/Production/Test Data/"

In [14]:
# Locate all the production ST files
gm175_ST1_prod_files, chrysler_ST1_prod_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST1_Prod_file_path, 0)
gm175_ST2_prod_files, chrysler_ST2_prod_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST2_Prod_file_path, 0)
gm175_ST3_prod_files, chrysler_ST3_prod_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST3_Prod_file_path, 0)
gm175_ST4_prod_files, chrysler_ST4_prod_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST4_Prod_file_path, 0)
gm175_ST5_prod_files, chrysler_ST5_prod_files = file_based_on_nest_RDR6_sensor_test(RDR6_ST5_Prod_file_path, 0)

In [75]:
# Process for GM175 ST production files
gm175_ST1_prod_df = combine_csv_RDR6_sensor_test(gm175_ST1_prod_files, info.ST_column_names_GM175_prod, info.ST_total_test_item_prod_RDR6_GM175, 0, "GM175")
gm175_ST2_prod_df = combine_csv_RDR6_sensor_test(gm175_ST2_prod_files, info.ST_column_names_GM175_prod, info.ST_total_test_item_prod_RDR6_GM175, 0, "GM175")
gm175_ST3_prod_df = combine_csv_RDR6_sensor_test(gm175_ST3_prod_files, info.ST_column_names_GM175_prod, info.ST_total_test_item_prod_RDR6_GM175, 0, "GM175")
gm175_ST4_prod_df = combine_csv_RDR6_sensor_test(gm175_ST4_prod_files, info.ST_column_names_GM175_prod, info.ST_total_test_item_prod_RDR6_GM175, 0, "GM175")
gm175_ST5_prod_df = combine_csv_RDR6_sensor_test(gm175_ST5_prod_files, info.ST_column_names_GM175_prod, info.ST_total_test_item_prod_RDR6_GM175, 0, "GM175")

In [13]:
# Add Nest column to each dataframe
gm175_ST1_prod_df["Tester_num"] = 1
gm175_ST2_prod_df["Tester_num"] = 2
gm175_ST3_prod_df["Tester_num"] = 3
gm175_ST4_prod_df["Tester_num"] = 4
gm175_ST5_prod_df["Tester_num"] = 5

In [14]:
# Form final dataframe
RDR6_ST_Prod_GM175 = pd.concat(objs = [gm175_ST1_prod_df, gm175_ST2_prod_df, gm175_ST3_prod_df, gm175_ST4_prod_df, gm175_ST5_prod_df], ignore_index = False)

In [15]:
# Output Log as csv
RDR6_ST_Prod_GM175.to_csv("../Data_clean/RDR6_ST_Prod_GM175.csv")

#### Chrysler Product

In [15]:
# Process for Chrysler ST production files
chrysler_ST1_prod_df = combine_csv_RDR6_sensor_test(chrysler_ST1_prod_files, info.ST_column_names_Chrysler_prod, info.ST_total_test_item_prod_RDR6_Chrysler, 0, "Chrysler")
chrysler_ST2_prod_df = combine_csv_RDR6_sensor_test(chrysler_ST2_prod_files, info.ST_column_names_Chrysler_prod, info.ST_total_test_item_prod_RDR6_Chrysler, 0, "Chrysler")
chrysler_ST3_prod_df = combine_csv_RDR6_sensor_test(chrysler_ST3_prod_files, info.ST_column_names_Chrysler_prod, info.ST_total_test_item_prod_RDR6_Chrysler, 0, "Chrysler")
chrysler_ST4_prod_df = combine_csv_RDR6_sensor_test(chrysler_ST4_prod_files, info.ST_column_names_Chrysler_prod, info.ST_total_test_item_prod_RDR6_Chrysler, 0, "Chrysler")
chrysler_ST5_prod_df = combine_csv_RDR6_sensor_test(chrysler_ST5_prod_files, info.ST_column_names_Chrysler_prod, info.ST_total_test_item_prod_RDR6_Chrysler, 0, "Chrysler")

In [17]:
# Add Nest column to each dataframe
chrysler_ST1_prod_df["Tester_num"] = 1
chrysler_ST2_prod_df["Tester_num"] = 2
chrysler_ST3_prod_df["Tester_num"] = 3
chrysler_ST4_prod_df["Tester_num"] = 4
chrysler_ST5_prod_df["Tester_num"] = 5

In [18]:
# Form final dataframe
RDR6_ST_Prod_Chrysler = pd.concat(objs = [chrysler_ST1_prod_df, chrysler_ST2_prod_df, chrysler_ST3_prod_df, chrysler_ST4_prod_df, chrysler_ST5_prod_df], ignore_index = False)

In [19]:
# Output Log as csv
RDR6_ST_Prod_Chrysler.to_csv("../Data_clean/RDR6_ST_Prod_Chrysler.csv")

# RDR7

## TxCal

### Golden

#### Honda1.0 Product

In [3]:
# Locate all the files
RDR7_TxCal_Golden_Honda_1_0_file_path = "../Final Project - Log Collection/RDR7/TxCal/Honda1.0/Golden/Honda_GOLD/"
Honda_1_0_golden_nest1, Honda_1_0_golden_nest2, Honda_1_0_golden_nest3, Honda_1_0_golden_nest4 = file_based_on_nest_RDR7_Txcal(RDR7_TxCal_Golden_Honda_1_0_file_path, 1)

In [10]:
# Process all the files
RDR7_Txcal_Golden_Honda_1_0_nest_1_df = combine_csv_RDR7_Txcal(Honda_1_0_golden_nest1, info.column_names_Honda_1_0_golden, info.total_test_item_golden_RDR7_Honda_1_0, info.RDR7_TxCal_golden_Honda_1_0_FID, info.RDR7_TxCal_golden_Honda_1_0_drop_index, 1, "1.0")
RDR7_Txcal_Golden_Honda_1_0_nest_2_df = combine_csv_RDR7_Txcal(Honda_1_0_golden_nest2, info.column_names_Honda_1_0_golden, info.total_test_item_golden_RDR7_Honda_1_0, info.RDR7_TxCal_golden_Honda_1_0_FID, info.RDR7_TxCal_golden_Honda_1_0_drop_index, 1, "1.0")
RDR7_Txcal_Golden_Honda_1_0_nest_3_df = combine_csv_RDR7_Txcal(Honda_1_0_golden_nest3, info.column_names_Honda_1_0_golden, info.total_test_item_golden_RDR7_Honda_1_0, info.RDR7_TxCal_golden_Honda_1_0_FID, info.RDR7_TxCal_golden_Honda_1_0_drop_index, 1, "1.0")
RDR7_Txcal_Golden_Honda_1_0_nest_4_df = combine_csv_RDR7_Txcal(Honda_1_0_golden_nest4, info.column_names_Honda_1_0_golden, info.total_test_item_golden_RDR7_Honda_1_0, info.RDR7_TxCal_golden_Honda_1_0_FID, info.RDR7_TxCal_golden_Honda_1_0_drop_index, 1, "1.0")

In [16]:
# Add Nest column to each dataframe
RDR7_Txcal_Golden_Honda_1_0_nest_1_df["Nest"] = 1
RDR7_Txcal_Golden_Honda_1_0_nest_2_df["Nest"] = 2
RDR7_Txcal_Golden_Honda_1_0_nest_3_df["Nest"] = 3
RDR7_Txcal_Golden_Honda_1_0_nest_4_df["Nest"] = 4

In [22]:
# Form final dataframe
RDR7_TxCal_Golden_Honda_1_0 = pd.concat(objs = [RDR7_Txcal_Golden_Honda_1_0_nest_1_df, RDR7_Txcal_Golden_Honda_1_0_nest_2_df, RDR7_Txcal_Golden_Honda_1_0_nest_3_df, RDR7_Txcal_Golden_Honda_1_0_nest_4_df], ignore_index = False)

In [24]:
# Output as csv
RDR7_TxCal_Golden_Honda_1_0.to_csv("../Data_clean/RDR7_TxCal_Golden_Honda_1_0.csv")

#### Honda 1.75 Product

In [3]:
RDR7_TxCal_Golden_Honda_175_file_path = "../Final Project - Log Collection/RDR7/TxCal/Honda1.75/Golden/Honda175_GOLD/"
Honda_175_golden_nest1, Honda_175_golden_nest2, Honda_175_golden_nest3, Honda_175_golden_nest4 = file_based_on_nest_RDR7_Txcal(RDR7_TxCal_Golden_Honda_175_file_path, 1)

In [4]:
# Process all the files
#RDR7_Txcal_Golden_Honda_175_nest_1_df = combine_csv_RDR7_Txcal(Honda_175_golden_nest1, info.column_names_Honda_175_golden, info.total_test_item_golden_RDR7_Honda_175, info.RDR7_TxCal_golden_Honda_175_FID, info.RDR7_TxCal_golden_Honda_175_drop_index, 1, "1.75")
RDR7_Txcal_Golden_Honda_175_nest_2_df = combine_csv_RDR7_Txcal(Honda_175_golden_nest2, info.column_names_Honda_175_golden, info.total_test_item_golden_RDR7_Honda_175, info.RDR7_TxCal_golden_Honda_175_FID, info.RDR7_TxCal_golden_Honda_175_drop_index, 1, "1.75")
RDR7_Txcal_Golden_Honda_175_nest_3_df = combine_csv_RDR7_Txcal(Honda_175_golden_nest3, info.column_names_Honda_175_golden, info.total_test_item_golden_RDR7_Honda_175, info.RDR7_TxCal_golden_Honda_175_FID, info.RDR7_TxCal_golden_Honda_175_drop_index, 1, "1.75")
RDR7_Txcal_Golden_Honda_175_nest_4_df = combine_csv_RDR7_Txcal(Honda_175_golden_nest4, info.column_names_Honda_175_golden, info.total_test_item_golden_RDR7_Honda_175, info.RDR7_TxCal_golden_Honda_175_FID, info.RDR7_TxCal_golden_Honda_175_drop_index, 1, "1.75")

In [5]:
# Add Nest column to each dataframe
#RDR7_Txcal_Golden_Honda_175_nest_1_df["Nest"] = 1
RDR7_Txcal_Golden_Honda_175_nest_2_df["Nest"] = 2
RDR7_Txcal_Golden_Honda_175_nest_3_df["Nest"] = 3
RDR7_Txcal_Golden_Honda_175_nest_4_df["Nest"] = 4

In [9]:
# Form final dataframe
RDR7_TxCal_Golden_Honda_175 = pd.concat(objs = [RDR7_Txcal_Golden_Honda_175_nest_2_df, RDR7_Txcal_Golden_Honda_175_nest_3_df, RDR7_Txcal_Golden_Honda_175_nest_4_df], ignore_index = False)

In [10]:
# Output as csv
RDR7_TxCal_Golden_Honda_175.to_csv("../Data_clean/RDR7_TxCal_Golden_Honda_175.csv")

### Production

#### Honda1.0 Product

In [3]:
# Locate all the files
RDR7_TxCal_Prod_Honda_1_0_file_path = "../Final Project - Log Collection/RDR7/TxCal/Honda1.0/Production/"

Honda_1_0_prod_nest1, Honda_1_0_prod_nest2, Honda_1_0_prod_nest3, Honda_1_0_prod_nest4, Honda_1_0_prod_test_result = file_based_on_nest_RDR7_Txcal(RDR7_TxCal_Prod_Honda_1_0_file_path, 0)

In [30]:
# Process all the files
RDR7_Txcal_Prod_Honda_1_0_nest_1_df = combine_csv_RDR7_Txcal(Honda_1_0_prod_nest1, info.column_names_Honda_1_0_prod, info.total_test_item_prod_RDR7_Honda_1_0, info.RDR7_TxCal_prod_Honda_1_0_FID, info.RDR7_TxCal_prod_Honda_1_0_drop_index, 0, "1.0")
RDR7_Txcal_Prod_Honda_1_0_nest_2_df = combine_csv_RDR7_Txcal(Honda_1_0_prod_nest2, info.column_names_Honda_1_0_prod, info.total_test_item_prod_RDR7_Honda_1_0, info.RDR7_TxCal_prod_Honda_1_0_FID, info.RDR7_TxCal_prod_Honda_1_0_drop_index, 0, "1.0")
RDR7_Txcal_Prod_Honda_1_0_nest_3_df = combine_csv_RDR7_Txcal(Honda_1_0_prod_nest3, info.column_names_Honda_1_0_prod, info.total_test_item_prod_RDR7_Honda_1_0, info.RDR7_TxCal_prod_Honda_1_0_FID, info.RDR7_TxCal_prod_Honda_1_0_drop_index, 0, "1.0")
RDR7_Txcal_Prod_Honda_1_0_nest_4_df = combine_csv_RDR7_Txcal(Honda_1_0_prod_nest4, info.column_names_Honda_1_0_prod, info.total_test_item_prod_RDR7_Honda_1_0, info.RDR7_TxCal_prod_Honda_1_0_FID, info.RDR7_TxCal_prod_Honda_1_0_drop_index, 0, "1.0")

In [36]:
# Due to the size of the log files, split the data process/upload into different years
prod_2018 = []
prod_2019 = []
prod_2020 = []
prod_2021 = []
for item in Honda_1_0_prod_test_result:
    if item.startswith("../Final Project - Log Collection/RDR7/TxCal/Honda1.0/Production/test results2018"):
        prod_2018.append(item)
    elif item.startswith("../Final Project - Log Collection/RDR7/TxCal/Honda1.0/Production/test results2019"):
        prod_2019.append(item)
    elif item.startswith("../Final Project - Log Collection/RDR7/TxCal/Honda1.0/Production/test results2020"):
        prod_2020.append(item)
    else:
        prod_2021.append(item)

In [ ]:
# Process for test result log files
# Initialization
column_names = info.RDR7_test_result_col_names
total_test_item_num = info.RDR7_total_test_result_item_num
final_data = pd.DataFrame(columns = ["Date", "Time", "Test Time", "Nest", "MIS", "Pass/Fail", "FailedTests"])

# for 2018
file_path = prod_2018
data_2018 = combine_test_result_Txcal(prod_2018, column_names, total_test_item_num)
final_data = pd.concat(objs = [final_data, data_2018], ignore_index = True )

# for 2019
file_path = prod_2019
data_2019 = combine_test_result_Txcal(prod_2019, column_names, total_test_item_num)
final_data = pd.concat(objs = [final_data, data_2019], ignore_index = True )

# for 2020
file_path = prod_2020
data_2020 = combine_test_result_Txcal(prod_2020, column_names, total_test_item_num)
final_data = pd.concat(objs = [final_data, data_2020], ignore_index = True )

# for 2021
file_path = prod_2021
data_2021 = combine_test_result_Txcal(prod_2021, column_names, total_test_item_num)
final_data = pd.concat(objs = [final_data, data_2021], ignore_index = True )

In [63]:
# Combine Test result files
Honda_1_0_prod_test_result_df = final_data

In [43]:
# Add Nest column to each dataframe
RDR7_Txcal_Prod_Honda_1_0_nest_1_df["Nest"] = 1
RDR7_Txcal_Prod_Honda_1_0_nest_2_df["Nest"] = 2
RDR7_Txcal_Prod_Honda_1_0_nest_3_df["Nest"] = 3
RDR7_Txcal_Prod_Honda_1_0_nest_4_df["Nest"] = 4

In [44]:
# Form final dataframe
RDR7_TxCal_Prod_Honda_1_0 = pd.concat(objs = [RDR7_Txcal_Prod_Honda_1_0_nest_1_df, RDR7_Txcal_Prod_Honda_1_0_nest_2_df, RDR7_Txcal_Prod_Honda_1_0_nest_3_df, RDR7_Txcal_Prod_Honda_1_0_nest_4_df], ignore_index = False)

In [45]:
# Output Log as csv
RDR7_TxCal_Prod_Honda_1_0.to_csv("../Data_clean/RDR7_TxCal_Prod_Honda_1_0.csv")

In [65]:
# Output test result as csv
Honda_1_0_prod_test_result_df.to_csv("../Data_clean/Honda_1_0_prod_test_result_df.csv", index = False)

#### Honda 1.75 Product

In [71]:
RDR7_TxCal_Prod_Honda_175_file_path = "../Final Project - Log Collection/RDR7/TxCal/Honda1.75/Production/Honda175/"
Honda_175_prod_nest1, Honda_175_prod_nest2, Honda_175_prod_nest3, Honda_175_prod_nest4, Honda_175_prod_test_result = file_based_on_nest_RDR7_Txcal(RDR7_TxCal_Prod_Honda_175_file_path, 0)

In [31]:
# Process all the files
#RRDR7_Txcal_Prod_Honda_175_nest_2_df = combine_csv_RDR7_Txcal(Honda_175_prod_nest2, info.column_names_Honda_175_prod, info.total_test_item_prod_RDR7_Honda_175, info.RDR7_TxCal_prod_Honda_175_FID, info.RDR7_TxCal_prod_Honda_175_drop_index, 0, "1.75")
RDR7_Txcal_Prod_Honda_175_nest_3_df = combine_csv_RDR7_Txcal(Honda_175_prod_nest3, info.column_names_Honda_175_prod, info.total_test_item_prod_RDR7_Honda_175, info.RDR7_TxCal_prod_Honda_175_FID, info.RDR7_TxCal_prod_Honda_175_drop_index, 0, "1.75")
RDR7_Txcal_Prod_Honda_175_nest_4_df = combine_csv_RDR7_Txcal(Honda_175_prod_nest4, info.column_names_Honda_175_prod, info.total_test_item_prod_RDR7_Honda_175, info.RDR7_TxCal_prod_Honda_175_FID, info.RDR7_TxCal_prod_Honda_175_drop_index, 0, "1.75")
RDR7_Txcal_Prod_Honda_175_nest_1_df = combine_csv_RDR7_Txcal(Honda_175_prod_nest1, info.column_names_Honda_175_prod, info.total_test_item_prod_RDR7_Honda_175, info.RDR7_TxCal_prod_Honda_175_FID, info.RDR7_TxCal_prod_Honda_175_drop_index, 0, "1.75")


In [73]:
# Combine Test result files
Honda_175_prod_test_result_df = combine_test_result_Txcal(Honda_175_prod_test_result, info.RDR7_test_result_col_names, info.RDR7_total_test_result_item_num)

In [75]:
# Add Nest column to each dataframe
#RDR7_Txcal_Prod_Honda_175_nest_1_df["Nest"] = 1
RDR7_Txcal_Prod_Honda_175_nest_2_df["Nest"] = 2
RDR7_Txcal_Prod_Honda_175_nest_3_df["Nest"] = 3
RDR7_Txcal_Prod_Honda_175_nest_4_df["Nest"] = 4

In [77]:
# Form final dataframe
RDR7_TxCal_Prod_Honda_175 = pd.concat(objs = [RDR7_Txcal_Prod_Honda_175_nest_2_df, RDR7_Txcal_Prod_Honda_175_nest_3_df, RDR7_Txcal_Prod_Honda_175_nest_4_df], ignore_index = False)

In [79]:
# Output as csv
RDR7_TxCal_Prod_Honda_175.to_csv("../Data_clean/RDR7_TxCal_Prod_Honda_175.csv")

In [80]:
# Output test result as csv
Honda_175_prod_test_result_df.to_csv("../Data_clean/Honda_175_prod_test_result_df.csv", index = False)

#### Combine to final log file

In [12]:
# For Honda1.0
# Prod
honda_1_0_df = pd.read_csv("../Data_clean/RDR7_TxCal_Prod_Honda_1_0.csv", parse_dates = ["Date"])
honda_1_0_df = merge_prod_test(honda_1_0_df)

# test result
honda_1_0_test = pd.read_csv("../Data_clean/Honda_1_0_prod_test_result_df.csv", parse_dates = ["Date"])
honda_1_0_test = merge_prod_test(honda_1_0_test)

# merge
honda_1_0_temp = honda_1_0_df.merge(honda_1_0_test, how = "left", left_index = True, right_index = True)
honda_1_0 = honda_1_0_temp[~honda_1_0_temp["Pass/Fail"].isnull()]
honda_1_0 = honda_1_0[["Date_Time", "Date_x", "Nest_x", "Time_x", "Test Time", "Pass/Fail", "FailedTests", "MIS_x", "Temperature", "RCTA Occupied BW", "Antenna1 GetTxPower", "Antenna2 GetTxPower", "Antenna1 GetCenterFrequency", "Antenna2 GetCenterFrequency", "Antenna1 GetCwLeakage", "Antenna2 GetCwLeakage"]]
honda_1_0_final = honda_1_0.rename(columns = {'Nest_x' : "Nest", "Time_x": "Time", "MIS_x" : "MIS"})


D:\Anaconda\software\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (30,31,32,33,34,40,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
D:\Anaconda\software\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
# fill NaN
honda_1_0_final = honda_1_0_final.fillna(value = "NA")

In [16]:
# Output (if needed) - Honda 1.0
honda_1_0_final.to_csv("../Data_clean/RDR7_TxCal_Prod_Honda_1_0_FINAL.csv", index = False)

In [29]:
# For Honda 1.75
# Prod
honda_175_df = pd.read_csv("../Data_clean/RDR7_TxCal_Prod_Honda_175.csv", parse_dates = ["Date"])
honda_175_df = merge_prod_test(honda_175_df)

# test result
honda_175_test = pd.read_csv("../Data_clean/Honda_175_prod_test_result_df.csv", parse_dates = ["Date"])
honda_175_test = merge_prod_test(honda_175_test)

# merge
honda_175_temp = honda_175_df.merge(honda_175_test, how = "left", left_index = True, right_index = True)
honda_175 = honda_175_temp[~honda_175_temp["Pass/Fail"].isnull()]
honda_175 = honda_175[["Date_Time", "Date_x", "Nest_x", "Time_x", "Test Time", "Pass/Fail", "MIS_x", "Temperature", "RCTA Occupied BW", "Antenna1 GetTxPower", "Antenna2 GetTxPower", "Antenna1 GetCenterFrequency", "Antenna2 GetCenterFrequency", "Antenna1 GetCwLeakage", "Antenna2 GetCwLeakage", "PrepareForPowerdown"]]
honda_175_final = honda_175.rename(columns = {'Nest_x' : "Nest", "Time_x": "Time", "MIS_x" : "MIS"})


In [31]:
# fill NaN
honda_175_final = honda_175_final.fillna(value = "NA")

In [32]:
# Output (if needed) - Honda 1.75
honda_175_final.to_csv("../Data_clean/RDR7_TxCal_Prod_Honda_175_FINAL.csv", index = False)

## Sensor Test

In [22]:
# Collect files based on nests
def file_based_on_nest_RDR7_sensor_test(file_path, golden_flag):
    
    # Get the list of all the file names
    files = os.listdir(file_path)
    
    # Init the file collection name
    honda_1_0_ST = []
    honda_175_ST = []
    
    if golden_flag:
        
        # Init dataframe and filter files based on the nest number in file names
        for file in files:
            if file.startswith("Golden"):
                honda_1_0_ST.append(file_path + file)
            elif file.startswith("22"):
                honda_175_ST.append(file_path + file)
    
    else:
        
        # Init dataframe and filter files based on the nest number in file names
        for file in files:
            if file.startswith("15") or file.startswith("74"):
                honda_1_0_ST.append(file_path + file)
            elif file.startswith("22"):
                honda_175_ST.append(file_path + file)
    
    return honda_1_0_ST, honda_175_ST


# Concatinate all dataframe based on nests
def combine_csv_RDR7_sensor_test(file_path, column_names, golden_flag, prod_type):
    
    # Initialize an empty dataframe for all files concatenation
    total_data = pd.DataFrame()
    col_final = column_names.split(",")

    # Concatenate all the csv files in the folder into one dataframe based on nest
    for item in file_path:
        file_temp = list()
        row_1_temp = list()
        max_col = 0
        with open(item, newline = '') as file:
            reader = csv.reader(file, delimiter = ",")
            try:
                row_1 = next(reader)
            except UnicodeDecodeError:
                print(item)
            file_temp.append(row_1)
            max_col = len(row_1)
            if prod_type == "Honda 1.0":
                if max_col < 140:
                    continue
            if prod_type == "Honda 1.75":
                if max_col < 45 or max_col > 200:
                    continue
                if "GM1.75" in str(row_1):
                    for item in row_1:
                        if "GM1.75" in item:
                            item = item.replace("GM1.75", "Honda1.75")
                        row_1_temp.append(item)
                    row_1 = row_1_temp
            row_1[0] = "Date"
            row_1[1] = "Time"
            row_1[2] = "SN"
            row_1[3] = "Nest"
            row_1[4] = "Pass/Fail"

            for row in reader:
                if len(row) == max_col:
                    file_temp.append(row)
            df = pd.DataFrame(file_temp, columns = row_1)
            df = df.loc[:,~df.columns.duplicated()]
            df = df[col_final]
            total_data = pd.concat(objs = [total_data, df], ignore_index = True)

    # Filter out test step name
    condition = total_data["Time"] != "Time"
    total_data = total_data[condition]
    total_data = total_data[pd.to_numeric(total_data["Nest"], errors = "coerce").notnull()]

    # Convert data type from string to float for plotting
    if prod_type == "Honda 1.0":
        total_data = total_data[pd.to_numeric(total_data["Normalized Tx2 Power @ 0"], errors = "coerce").notnull()]
        total_data = total_data[pd.to_numeric(total_data["RCTA Main SNR @ 0"], errors = "coerce").notnull()]
        clean_data = total_data[pd.to_numeric(total_data["Tx1 EIRP"], errors = "coerce").notnull()]

        clean_data["Normalized Tx2 Power @ 0"] = clean_data["Normalized Tx2 Power @ 0"].astype("float")
        clean_data["RCTA Main SNR @ 0"] = clean_data["RCTA Main SNR @ 0"].astype("float")
        clean_data["Tx1 EIRP"] = clean_data["Tx1 EIRP"].astype("float")

    else:
        total_data = total_data[pd.to_numeric(total_data["Honda1.75 Angle @ -30"], errors = "coerce").notnull()]
        total_data = total_data[pd.to_numeric(total_data["Honda1.75 Angle @ 0"], errors = "coerce").notnull()]
        total_data = total_data[pd.to_numeric(total_data["Honda1.75 Angle @ 30"], errors = "coerce").notnull()]
        total_data = total_data[pd.to_numeric(total_data["LCA Main SNR @ -30"], errors = "coerce").notnull()]
        clean_data = total_data[pd.to_numeric(total_data["LCA Main SNR @ 30"], errors = "coerce").notnull()]

        clean_data["Honda1.75 Angle @ -30"] = clean_data["Honda1.75 Angle @ -30"].astype("float")
        clean_data["Honda1.75 Angle @ 0"] = clean_data["Honda1.75 Angle @ 0"].astype("float")
        clean_data["Honda1.75 Angle @ 30"] = clean_data["Honda1.75 Angle @ 30"].astype("float")
        clean_data["LCA Main SNR @ -30"] = clean_data["LCA Main SNR @ -30"].astype("float")
        clean_data["LCA Main SNR @ 30"] = clean_data["LCA Main SNR @ 30"].astype("float")

    # Reset index column
    clean_data = clean_data.reset_index(drop = True)

    # Convert other data type
    clean_data["SN"] = clean_data["SN"].fillna("000000")

    # Add Date_time column and set as index column
    clean_data["Date_Time"] = clean_data["Date"] + " " + clean_data["Time"]
    clean_data = clean_data.set_index(["Date_Time"])
    
    return clean_data

### Golden

#### Honda 1.0 Product

In [5]:
# Sensor Test golden files directory
RDR7_ST1_Golden_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST1/Golden/Golden/"
RDR7_ST2_Golden_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST2/Golden/Golden/"
RDR7_ST3_Golden_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST3/Golden/Golden/"
RDR7_ST4_Golden_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST4/Golden/Golden/"
RDR7_ST5_Golden_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST5/Golden/Golden/"

In [6]:
# Locate all the golden ST files
Honda_1_0_ST1_golden_files, Honda_175_ST1_golden_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST1_Golden_file_path, 1)
Honda_1_0_ST2_golden_files, Honda_175_ST2_golden_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST2_Golden_file_path, 1)
Honda_1_0_ST3_golden_files, Honda_175_ST3_golden_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST3_Golden_file_path, 1)
Honda_1_0_ST4_golden_files, Honda_175_ST4_golden_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST4_Golden_file_path, 1)
Honda_1_0_ST5_golden_files, Honda_175_ST5_golden_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST5_Golden_file_path, 1)

In [16]:
# Process for Honda 1.0 ST golden files
Honda_1_0_ST1_golden_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST1_golden_files, info.ST_column_names_Honda_1_0_golden, 1, "Honda 1.0")
Honda_1_0_ST2_golden_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST2_golden_files, info.ST_column_names_Honda_1_0_golden, 1, "Honda 1.0")
Honda_1_0_ST3_golden_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST3_golden_files, info.ST_column_names_Honda_1_0_golden, 1, "Honda 1.0")
Honda_1_0_ST4_golden_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST4_golden_files, info.ST_column_names_Honda_1_0_golden, 1, "Honda 1.0")
Honda_1_0_ST5_golden_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST5_golden_files, info.ST_column_names_Honda_1_0_golden, 1, "Honda 1.0")

In [17]:
# Add Nest column to each dataframe
Honda_1_0_ST1_golden_df["Tester_num"] = 1
Honda_1_0_ST2_golden_df["Tester_num"] = 2
Honda_1_0_ST3_golden_df["Tester_num"] = 3
Honda_1_0_ST4_golden_df["Tester_num"] = 4
Honda_1_0_ST5_golden_df["Tester_num"] = 5

In [18]:
# Form final dataframe
RDR7_ST_Golden_Honda_1_0 = pd.concat(objs = [Honda_1_0_ST1_golden_df, Honda_1_0_ST2_golden_df, Honda_1_0_ST3_golden_df, Honda_1_0_ST4_golden_df, Honda_1_0_ST5_golden_df], ignore_index = False)

In [19]:
# Output Log as csv
RDR7_ST_Golden_Honda_1_0.to_csv("../Data_clean/RDR7_ST_Golden_Honda_1_0.csv")

#### Honda 1.75 Product

In [29]:
# Process for Chrysler ST golden files
Honda_175_ST1_golden_df = combine_csv_RDR7_sensor_test(Honda_175_ST1_golden_files, info.ST_column_names_Honda_175_golden, 1, "Honda 1.75")
Honda_175_ST2_golden_df = combine_csv_RDR7_sensor_test(Honda_175_ST2_golden_files, info.ST_column_names_Honda_175_golden, 1, "Honda 1.75")
Honda_175_ST3_golden_df = combine_csv_RDR7_sensor_test(Honda_175_ST3_golden_files, info.ST_column_names_Honda_175_golden, 1, "Honda 1.75")
Honda_175_ST4_golden_df = combine_csv_RDR7_sensor_test(Honda_175_ST4_golden_files, info.ST_column_names_Honda_175_golden, 1, "Honda 1.75")
Honda_175_ST5_golden_df = combine_csv_RDR7_sensor_test(Honda_175_ST5_golden_files, info.ST_column_names_Honda_175_golden, 1, "Honda 1.75")

In [38]:
# Add Nest column to each dataframe
Honda_175_ST1_golden_df["Tester_num"] = 1
Honda_175_ST2_golden_df["Tester_num"] = 2
Honda_175_ST3_golden_df["Tester_num"] = 3
Honda_175_ST4_golden_df["Tester_num"] = 4
Honda_175_ST5_golden_df["Tester_num"] = 5

In [39]:
# Form final dataframe
RDR7_ST_Golden_Honda_175 = pd.concat(objs = [Honda_175_ST1_golden_df, Honda_175_ST2_golden_df, Honda_175_ST3_golden_df, Honda_175_ST4_golden_df, Honda_175_ST5_golden_df], ignore_index = False)

In [40]:
# Output Log as csv
RDR7_ST_Golden_Honda_175.to_csv("../Data_clean/RDR7_ST_Golden_Honda_175.csv")

## Production

### Honda 1.0

In [23]:
# Sensor Test production files directory
RDR7_ST1_Prod_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST1/Production/Test Data/"
RDR7_ST2_Prod_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST2/Production/Test Data/"
RDR7_ST3_Prod_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST3/Production/Test Data/"
RDR7_ST4_Prod_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST4/Production/Test Data/"
RDR7_ST5_Prod_file_path = "../Final Project - Log Collection/RDR7/Sensor Test/ST5/Production/Test Data/"

In [24]:
# Locate all the production ST files
Honda_1_0_ST1_prod_files, Honda_175_ST1_prod_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST1_Prod_file_path, 0)
Honda_1_0_ST2_prod_files, Honda_175_ST2_prod_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST2_Prod_file_path, 0)
Honda_1_0_ST3_prod_files, Honda_175_ST3_prod_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST3_Prod_file_path, 0)
Honda_1_0_ST4_prod_files, Honda_175_ST4_prod_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST4_Prod_file_path, 0)
Honda_1_0_ST5_prod_files, Honda_175_ST5_prod_files = file_based_on_nest_RDR7_sensor_test(RDR7_ST5_Prod_file_path, 0)

In [16]:
# Process for Honda 1.0 ST production files
Honda_1_0_ST1_prod_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST1_prod_files, info.ST_column_names_Honda_1_0_prod, 0, "Honda 1.0")
Honda_1_0_ST2_prod_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST2_prod_files, info.ST_column_names_Honda_1_0_prod, 0, "Honda 1.0")
Honda_1_0_ST3_prod_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST3_prod_files, info.ST_column_names_Honda_1_0_prod, 0, "Honda 1.0")
Honda_1_0_ST4_prod_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST4_prod_files, info.ST_column_names_Honda_1_0_prod, 0, "Honda 1.0")
Honda_1_0_ST5_prod_df = combine_csv_RDR7_sensor_test(Honda_1_0_ST5_prod_files, info.ST_column_names_Honda_1_0_prod, 0, "Honda 1.0")

In [13]:
# Add Nest column to each dataframe
Honda_1_0_ST1_prod_df["Tester_num"] = 1
Honda_1_0_ST2_prod_df["Tester_num"] = 2
Honda_1_0_ST3_prod_df["Tester_num"] = 3
Honda_1_0_ST4_prod_df["Tester_num"] = 4
Honda_1_0_ST5_prod_df["Tester_num"] = 5

In [14]:
# Form final dataframe
RDR7_ST_Prod_Honda_1_0 = pd.concat(objs = [Honda_1_0_ST1_prod_df, Honda_1_0_ST2_prod_df, Honda_1_0_ST3_prod_df, Honda_1_0_ST4_prod_df, Honda_1_0_ST5_prod_df], ignore_index = False)

In [15]:
# Output Log as csv
RDR7_ST_Prod_Honda_1_0.to_csv("../Data_clean/RDR7_ST_Prod_Honda_1_0.csv")

#### Honda1.75 Product

In [25]:
# Process for Chrysler ST production files
Honda_175_ST1_prod_df = combine_csv_RDR7_sensor_test(Honda_175_ST1_prod_files, info.ST_column_names_Honda_175_prod, 0, "Honda 1.75")
Honda_175_ST2_prod_df = combine_csv_RDR7_sensor_test(Honda_175_ST2_prod_files, info.ST_column_names_Honda_175_prod, 0, "Honda 1.75")
Honda_175_ST3_prod_df = combine_csv_RDR7_sensor_test(Honda_175_ST3_prod_files, info.ST_column_names_Honda_175_prod, 0, "Honda 1.75")
Honda_175_ST4_prod_df = combine_csv_RDR7_sensor_test(Honda_175_ST4_prod_files, info.ST_column_names_Honda_175_prod, 0, "Honda 1.75")
Honda_175_ST5_prod_df = combine_csv_RDR7_sensor_test(Honda_175_ST5_prod_files, info.ST_column_names_Honda_175_prod, 0, "Honda 1.75")

In [26]:
# Add Nest column to each dataframe
Honda_175_ST1_prod_df["Tester_num"] = 1
Honda_175_ST2_prod_df["Tester_num"] = 2
Honda_175_ST3_prod_df["Tester_num"] = 3
Honda_175_ST4_prod_df["Tester_num"] = 4
Honda_175_ST5_prod_df["Tester_num"] = 5

In [27]:
# Form final dataframe
RDR7_ST_Prod_Honda_175 = pd.concat(objs = [Honda_175_ST1_prod_df, Honda_175_ST2_prod_df, Honda_175_ST3_prod_df, Honda_175_ST4_prod_df, Honda_175_ST5_prod_df], ignore_index = False)

In [28]:
# Output Log as csv
RDR7_ST_Prod_Honda_175.to_csv("../Data_clean/RDR7_ST_Prod_Honda_175.csv")

# Load data into AWS RDS

In [44]:
# Load data into SQL
def load_data_to_sql(csv_path, table_name, server_name):
    # Create a link for connection
    link_sql = f"postgres://{cd.sql_username}:{cd.sql_password}@{cd.sql_host}:{cd.sql_port}/{server_name}"

    # create a engine instance
    engine = create_engine(link_sql)
    
    # Start to load while recording time
    start_time = time.time()
    for data in pd.read_csv(csv_path, chunksize = 1000000):
        data.to_sql(name = table_name, con = engine, if_exists = "append")
        end_time = time.time()
        time_elapsed = end_time - start_time
        print(time_elapsed)


In [45]:
# Collect all the files and table names
file_path = "../Data_clean/"
#files = os.listdir(file_path)
files = ['RDR7_ST_Prod_Honda_1_0.csv', 'RDR7_TxCal_Golden_Honda_175.csv', 'RDR7_TxCal_Golden_Honda_1_0.csv', 'RDR7_TxCal_Prod_Honda_175.csv', 'RDR7_TxCal_Prod_Honda_1_0.csv']
table_name = []
for file in files:
    table_name.append(file[:-4])

In [1]:
# Load data into SQL database
for table in table_name:
    path = file_path + table + ".csv"
    server = table[:4]
    load_data_to_sql(path, table, server)

# Pull data from AWS RDS database

In [28]:
# Pull data from the last month only for demonstration (past 5 weeks)
df = pd.read_csv("../Data_clean/RDR6_TxCal_Prod_GM175.csv")
df["TestDate"] = pd.to_datetime(df["TestDate"])
time_end = df.iloc[-1, :]["TestDate"]
one_month = pd.Timedelta(weeks = 5)
time_start = time_end - one_month
time_start_str = time_start.strftime("%Y-%m-%d")

In [29]:
# Use the "last month timestamp" to extract data using Python from AWS RDS
connection = psycopg2.connect(user = cd.sql_username,
                             password = cd.sql_password,
                             host = cd.sql_host,
                             port = cd.sql_port,
                             database = "RDR6")
cursor = connection.cursor()
postgreSQL_query = f'SELECT * FROM public."RDR6_TxCal_Prod_GM175" WHERE "TestDate" > \'{time_start_str}\''
cursor.execute(postgreSQL_query)
SQL_record = cursor.fetchall()

In [35]:
# Get the column names for the new dataframe
with open("../Data_clean/RDR6_TxCal_Prod_GM175.csv", "r") as file:
    reader = csv.reader(file, delimiter = ",")
    sql_col_names = next(reader)
sql_col_names.insert(0, "index")

# Convert to JSON file for MongoDB storage and Web development

In [40]:
# Form a dataframe for JSON file conversion
sql_df = pd.DataFrame(SQL_record, columns = sql_col_names)
sql_df = sql_df.drop(["index"], axis = 1)

,Date_Time,TestDate,Nest,Time,Test Time,Pass/Fail,Current,MIS,FID,Temperature,RCTA Occupied BW,Tx1 BSD,Tx1 RCTA,Tx2 BSD,Tx2 RCTA,Antenna1 GetTxPower,Antenna2 GetTxPower,Antenna1 GetCenterFrequency,Antenna2 GetCenterFrequency
0,2021-04-14 21:53:24,2021-04-14,6,21:53:24,36.5,PASS,295.481,144521034231,1445,31.70032,Result= 0 Bandwidth=141.823444 MHz Min=None Ma...,13.491488,15.365446,13.417552,15.295464,15.459028,15.330523,24.059223,24.059216
1,2021-04-14 21:53:44,2021-04-14,2,21:53:44,36.3,PASS,273.837,144521034232,1445,30.96006,Result= 0 Bandwidth=160.347323 MHz Min=None Ma...,13.337000,15.356859,13.225157,15.379736,15.404438,15.458447,24.059520,24.059511
2,2021-04-14 21:53:56,2021-04-14,4,21:53:56,34.2,PASS,297.757,144521034233,1445,31.17597,Result= 0 Bandwidth=159.768452 MHz Min=None Ma...,13.456238,15.391927,13.246805,15.354334,15.415989,15.414852,24.059677,24.059671
3,2021-04-14 21:54:49,2021-04-14,1,21:54:49,36.3,PASS,276.767,144521034235,1445,31.39188,Result= 0 Bandwidth=145.296672 MHz Min=None Ma...,13.477686,15.357753,13.494008,15.341589,15.514202,15.478959,24.059737,24.059729
4,2021-04-14 21:54:57,2021-04-14,3,21:54:57,37.9,PASS,290.110,144521034236,1445,31.23766,Result= 0 Bandwidth=158.610709 MHz Min=None Ma...,13.300534,15.382627,13.315097,15.314523,15.451554,15.494834,24.059323,24.059317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14693,2021-05-18 00:05:44,2021-05-18,4,00:05:44,36.9,PASS,295.407,144420399503,1444,30.06558,Result= 0 Bandwidth=159.768451 MHz Min=None Ma...,13.263289,15.336131,13.299018,15.377627,15.443288,15.495537,24.059724,24.059718
14694,2021-05-18 00:06:21,2021-05-18,6,00:06:21,34.2,PASS,290.733,144420399504,1444,29.97305,Result= 0 Bandwidth=148.191027 MHz Min=None Ma...,13.492223,15.234127,13.364986,15.296549,15.312619,15.391666,24.059252,24.059245
14695,2021-05-18 06:14:49,2021-05-18,2,06:14:49,35.4,PASS,278.227,144420402672,1444,28.83182,Result= 0 Bandwidth=147.033285 MHz Min=None Ma...,13.295517,15.361637,13.472017,15.155410,15.436445,15.257065,24.059876,24.059867
14696,2021-05-18 06:14:47,2021-05-18,4,06:14:47,36.8,PASS,272.137,144420402673,1444,29.51039,Result= 0 Bandwidth=151.085384 MHz Min=None Ma...,13.175946,15.263234,13.455581,15.312643,15.312608,15.474814,24.059853,24.059842


In [42]:
# Export to JSON file
df_dict = sql_df.to_dict('index')
with open("RDR6_Txcal_Prod.json", "w") as outfile:
    json.dump(df_dict, outfile)

# Load data into MongoDB

In [43]:
# Create connection
Client = MongoClient("mongodb://localhost:27017/Veoneer_Data_Project")

# Database
db = Client["Veoneer_Data_Project"]

# Create collection
Collection = db["RDR6_TxCal_Prod"]

# Loading the json file into MongoDB
with open('RDR6_Txcal_Prod.json') as file:
    file_data = json.load(file)
if isinstance(file_data, list):
    Collection.insert_many(file_data)
else:
    Collection.insert_one(file_data)

# Use Flask for web development

# For Test

In [56]:
df_gm175 = pd.read_csv("Data_clean/RDR6_TxCal_Prod_GM175.csv")

D:\Anaconda\software\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12,17,21,24,51,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date_Time,TestDate,Time,Bat_Ctrl,Current,InitCanInterface,Sensor_Comm_Init,Get_Test_Instrument,Init_Tx_Cal,PingDut,...,Antenna2 GetTxPower,Antenna1 GetCenterFrequency,Antenna2 GetCenterFrequency,Antenna1 GetCwLeakage,Antenna2 GetCwLeakage,Delta,SerialpointsCheck,ReadPointCSV,VerifyNBMailboxPoints,Nest


In [60]:
df_gm175.info(0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315604 entries, 0 to 315603
Data columns (total 67 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Date_Time                    315604 non-null  object 
 1   TestDate                     315604 non-null  object 
 2   Time                         315604 non-null  object 
 3   Bat_Ctrl                     315604 non-null  int64  
 4   Current                      315604 non-null  float64
 5   InitCanInterface             315604 non-null  object 
 6   Sensor_Comm_Init             315604 non-null  object 
 7   Get_Test_Instrument          315604 non-null  int64  
 8   Init_Tx_Cal                  315604 non-null  object 
 9   PingDut                      315604 non-null  object 
 10  MIS                          315604 non-null  int64  
 11  FID                          315604 non-null  int64  
 12  Point 1571                   315604 non-null  object 
 13 

In [58]:
df = pd.read_csv("Data_clean/RDR6_TxCal_Prod_Chrysler.csv")

D:\Anaconda\software\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,15,16,17,18,19,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733422 entries, 0 to 733421
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Date_Time                    733422 non-null  object 
 1   TestDate                     733422 non-null  object 
 2   Time                         733422 non-null  object 
 3   MIS                          733422 non-null  int64  
 4   Point 1571                   733422 non-null  object 
 5   Point 1572                   733422 non-null  object 
 6   Point 1573                   733422 non-null  object 
 7   Point 1574                   733422 non-null  int64  
 8   Point 1575                   733422 non-null  int64  
 9   Point 1576                   733422 non-null  float64
 10  Temperature                  733422 non-null  float64
 11  Current                      733422 non-null  float64
 12  RCTA Occupied BW             733422 non-null  float64
 13 

In [61]:
df["MIS"] = df["MIS"].astype(str)

In [62]:
df["Nest"] = df["Nest"].astype(str)

In [63]:
df

,Date_Time,TestDate,Time,MIS,Point 1571,Point 1572,Point 1573,Point 1574,Point 1575,Point 1576,...,Point 1668,Point 1669,Antenna1 GetTxPower,Antenna2 GetTxPower,Antenna1 GetCenterFrequency,Antenna2 GetCenterFrequency,Antenna1 GetCwLeakage,Antenna2 GetCwLeakage,Delta,Nest
0,2017-01-02 07:06:11,2017-01-02,07:06:11,171680310936,62347,34BAA,17W01,13171,68031,936.0,...,15,4,12.421031,6.430558,24.201431,24.201430,28.0312,25.8463,6.135310,1
1,2017-01-02 23:27:16,2017-01-02,23:27:16,171680310916,62347,34BAA,17W01,13171,68031,916.0,...,11,4,12.292318,6.974748,24.201332,24.201331,27.1945,25.2877,5.827150,1
2,2017-01-02 23:50:56,2017-01-02,23:50:56,171680310951,62347,34BAA,17W01,13171,68031,951.0,...,11,4,12.300733,6.501974,24.201337,24.201336,26.2765,24.7661,5.827140,1
3,2017-01-03 00:07:15,2017-01-03,00:07:15,171680310954,62347,34BAA,17W01,23171,68031,954.0,...,8,4,12.022845,6.042576,24.201434,24.201436,26.1587,23.7584,5.631040,1
4,2017-01-03 00:12:07,2017-01-03,00:12:07,171680310956,62347,34BAA,17W01,23171,68031,956.0,...,22,4,12.481338,7.113448,24.201315,24.201315,27.5728,25.7489,6.527510,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733417,2021-02-19 21:43:30,2021-02-19,21:43:30,173780198396,68014,9500B,21W08,53173,78019,8396.0,...,11,4,12.309621,7.952698,24.167501,24.167500,28.2116,26.6243,5.71508,6
733418,2021-02-19 21:44:23,2021-02-19,21:44:23,173780198423,68014,9500B,21W08,53173,78019,8423.0,...,12,4,12.470383,9.131708,24.167515,24.167514,26.946,25.9458,5.68706,6
733419,2021-02-19 21:45:20,2021-02-19,21:45:20,173780198425,68014,9500B,21W08,53173,78019,8425.0,...,11,4,12.246765,8.577666,24.167546,24.167547,28.1586,26.9137,5.49095,6
733420,2021-02-19 21:51:40,2021-02-19,21:51:40,173780198434,68014,9500B,21W08,53173,78019,8434.0,...,11,4,12.334269,8.596407,24.167455,24.167454,30.4367,28.6717,5.68706,6


In [64]:
df["Identity"] = df["TestDate"] + "_" + df["MIS"] + "_" + df["Nest"]

In [65]:
df

,Date_Time,TestDate,Time,MIS,Point 1571,Point 1572,Point 1573,Point 1574,Point 1575,Point 1576,...,Point 1669,Antenna1 GetTxPower,Antenna2 GetTxPower,Antenna1 GetCenterFrequency,Antenna2 GetCenterFrequency,Antenna1 GetCwLeakage,Antenna2 GetCwLeakage,Delta,Nest,Identity
0,2017-01-02 07:06:11,2017-01-02,07:06:11,171680310936,62347,34BAA,17W01,13171,68031,936.0,...,4,12.421031,6.430558,24.201431,24.201430,28.0312,25.8463,6.135310,1,2017-01-02_171680310936_1
1,2017-01-02 23:27:16,2017-01-02,23:27:16,171680310916,62347,34BAA,17W01,13171,68031,916.0,...,4,12.292318,6.974748,24.201332,24.201331,27.1945,25.2877,5.827150,1,2017-01-02_171680310916_1
2,2017-01-02 23:50:56,2017-01-02,23:50:56,171680310951,62347,34BAA,17W01,13171,68031,951.0,...,4,12.300733,6.501974,24.201337,24.201336,26.2765,24.7661,5.827140,1,2017-01-02_171680310951_1
3,2017-01-03 00:07:15,2017-01-03,00:07:15,171680310954,62347,34BAA,17W01,23171,68031,954.0,...,4,12.022845,6.042576,24.201434,24.201436,26.1587,23.7584,5.631040,1,2017-01-03_171680310954_1
4,2017-01-03 00:12:07,2017-01-03,00:12:07,171680310956,62347,34BAA,17W01,23171,68031,956.0,...,4,12.481338,7.113448,24.201315,24.201315,27.5728,25.7489,6.527510,1,2017-01-03_171680310956_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733417,2021-02-19 21:43:30,2021-02-19,21:43:30,173780198396,68014,9500B,21W08,53173,78019,8396.0,...,4,12.309621,7.952698,24.167501,24.167500,28.2116,26.6243,5.71508,6,2021-02-19_173780198396_6
733418,2021-02-19 21:44:23,2021-02-19,21:44:23,173780198423,68014,9500B,21W08,53173,78019,8423.0,...,4,12.470383,9.131708,24.167515,24.167514,26.946,25.9458,5.68706,6,2021-02-19_173780198423_6
733419,2021-02-19 21:45:20,2021-02-19,21:45:20,173780198425,68014,9500B,21W08,53173,78019,8425.0,...,4,12.246765,8.577666,24.167546,24.167547,28.1586,26.9137,5.49095,6,2021-02-19_173780198425_6
733420,2021-02-19 21:51:40,2021-02-19,21:51:40,173780198434,68014,9500B,21W08,53173,78019,8434.0,...,4,12.334269,8.596407,24.167455,24.167454,30.4367,28.6717,5.68706,6,2021-02-19_173780198434_6


In [26]:
file_path = Honda_1_0_golden_nest1
column_names = info.column_names_Honda_1_0_golden
total_test_item_num = info.total_test_item_golden_RDR7_Honda_1_0
Golden_FID = info.RDR7_TxCal_golden_Honda_1_0_FID

In [27]:
# Clean the column names (remove extra space)
column_names_list = column_names.split(",")
col_name_clean = []
for i in column_names_list:
    col_name_clean.append(i.strip())

In [59]:
file_example = ['Final Project - Log Collection/RDR7/TxCal/Honda1.0/Golden/Honda_GOLD/TestOutput_Nest1_20180419.csv',
 'Final Project - Log Collection/RDR7/TxCal/Honda1.0/Golden/Honda_GOLD/TestOutput_Nest1_20180420.csv',
 'Final Project - Log Collection/RDR7/TxCal/Honda1.0/Golden/Honda_GOLD/TestOutput_Nest1_20180423.csv',
 'Final Project - Log Collection/RDR7/TxCal/Honda1.0/Golden/Honda_GOLD/TestOutput_Nest1_20180424.csv']

In [60]:
# Initialize an empty dataframe for all files concatenation
column_name_temp = list(range(100))
total_data = pd.DataFrame()

for item in file_example:
    csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
    csv_file_sample = csv_file_sample.drop([13, 18], axis = 1)
    total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = True)

# Eliminate redundant/skewed data
total_data = total_data.iloc[:, :28]

In [61]:
total_data

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-04-20,00:04:33,NaN,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,FID,Make_Path,SetTemperatureCompensation,Temperature,SetCWMode,Select_TX_Antenna,WaitForTemp,Antenna1 GetTxPower,SetCWMode,Bat_Ctrl State
3,2018-04-20,15:36:56,000080076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,0000,Error = 0,Error = 0,25.20437,Mode=1,Error = 0,30,13.397572,Mode=0,0
4,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,FID,Make_Path,SetTemperatureCompensation,Temperature,SetCWMode,Select_TX_Antenna,WaitForTemp,Antenna1 GetTxPower,SetCWMode,Bat_Ctrl State
5,2018-04-23,07:25:34,000080076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,0000,Error = 0,Error = 0,30.8354,Mode=1,Error = 0,30,13.299578,Mode=0,0
6,2018-04-23,15:48:02,000080076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,0000,Error = 0,Error = 0,24.39193,Mode=1,Error = 0,30,13.362424,Mode=0,0
7,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,FID,Make_Path,SetTemperatureCompensation,Temperature,SetCWMode,Select_TX_Antenna,WaitForTemp,Antenna1 GetTxPower,SetCWMode,Bat_Ctrl State
8,2018-04-24,09:23:14,000080076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,0000,Error = 0,Error = 0,30.89143,Mode=1,Error = 0,30,13.319383,Mode=0,0
9,2018-04-24,15:32:19,000080076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,0000,Error = 0,Error = 0,23.99971,Mode=1,Error = 0,30,13.329401,Mode=0,0


In [46]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-28,09:15:29,999980076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-28,17:03:13,999980076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df = df.drop([13, 18], axis = 1)
df = df.dropna(how = "all", axis = 1)

In [48]:
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,FID,Make_Path,SetTemperatureCompensation,Temperature,SetCWMode,Select_TX_Antenna,WaitForTemp,Antenna1 GetTxPower,SetCWMode,Bat_Ctrl State
1,2020-02-28,09:15:29,999980076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,9999,Error = 0,Error = 0,23.10323,Mode=1,Error = 0,30,12.988552,Mode=0,0
2,2020-02-28,17:03:13,999980076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,9999,Error = 0,Error = 0,23.71957,Mode=1,Error = 0,30,13.034925,Mode=0,0


In [33]:
df.columns = col_name_clean

In [34]:
df

,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,FID,Make_Path,SetTemperatureCompensation,Temperature,SetCWMode,Select_TX_Antenna,WaitForTemp,Antenna1 GetTxPower,SetCWMode,Bat_Ctrl State
0,Date,Time,MIS,Bat_Ctrl State,InitCanInterface,Sensor_Comm_Init,Init_Tx_Cal,PingDut,ReadGoldDACValError,1666,...,FID,Make_Path,SetTemperatureCompensation,Temperature,SetCWMode,Select_TX_Antenna,WaitForTemp,Antenna1 GetTxPower,SetCWMode,Bat_Ctrl State
1,2020-02-28,09:15:29,999980076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,9999,Error = 0,Error = 0,23.10323,Mode=1,Error = 0,30,12.988552,Mode=0,0
2,2020-02-28,17:03:13,999980076565,1,Connection_Status=0,Sensor_Comm_Init=0,InitTxCal=0,PING=1,0,8,...,9999,Error = 0,Error = 0,23.71957,Mode=1,Error = 0,30,13.034925,Mode=0,0


In [6]:
df_result = pd.read_csv("Final Project - Log Collection/RDR6/TxCal/Production/GM175/GM175 production log/test results.csv")

In [8]:
df_result = df_result.iloc[:, :6]

In [10]:
df_result.columns = 

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Name
0,NaN,NaN,NaN,NaN,NaN,Procedure
1,Date,Time,Test Time,Nest,MIS#,Pass/Fail
2,05-18-2021,00:00:17,33.1,1,144420399261,PASS
3,05-18-2021,00:00:51,33.4,2,144420399262,PASS
4,05-18-2021,00:01:50,34.7,4,144420399263,PASS
5,05-18-2021,00:02:52,34.4,6,144420399264,PASS
6,05-18-2021,00:03:18,32.3,1,144420399265,PASS
7,05-18-2021,00:03:52,34.5,2,144420399266,PASS
8,05-18-2021,00:03:53,36.9,4,144420399451,PASS
9,05-18-2021,00:04:21,35.4,6,144420399452,PASS


In [6]:
file_path = chrysler_prod_nest6
column_names = info.column_names_GM175_prod
total_test_item_num = info.total_test_item_prod_RDR6_GM175
Golden_FID = info.RDR6_TxCal_prod_GM175_FID

# Clean the column names (remove extra space)
column_names_list = column_names.split(",")
col_name_clean = []
for i in column_names_list:
    col_name_clean.append(i.strip())

# Initialize an empty dataframe for all files concatenation
column_name_temp = list(range(100))
total_data = pd.DataFrame(columns = column_name_temp)

# Concatenate all the csv files in the folder into one dataframe based on nest
for item in file_path:
    try:
        csv_file_sample = pd.read_csv(item, sep = ',', names = column_name_temp)
    except UnicodeDecodeError:
        print(item)
        
    total_data = pd.concat(objs = [total_data, csv_file_sample], ignore_index = True)

# Eliminate redundant/skewed data
total_data = total_data.iloc[:, :total_test_item_num]

# Name columns
total_data.columns = col_name_clean

# Filter out test step name
condition = total_data["Time"] != "Time"
total_data = total_data[condition]

# Filter out non-golden test records
if Golden_FID == "0000":
    clean_data = total_data
else:
    clean_data = total_data[total_data["FID"] == Golden_FID]

# Drop duplicated columns
clean_data = clean_data.loc[:,~clean_data.columns.duplicated()]

# Convert data type from string to float for plotting
try:
    clean_data["Antenna1 GetTxPower"] = clean_data["Antenna1 GetTxPower"].astype("float")
except ValueError:
    for item in clean_data["Antenna1 GetTxPower"]:
        try:
            item = float(item)
        except ValueError:
            drop_index_1 = clean_data[clean_data["Antenna1 GetTxPower"] == item].index
            clean_data = clean_data.drop(drop_index_1, axis = 0)

try:    
    clean_data["Temperature"] = clean_data["Temperature"].astype("float")
except ValueError:
    for item in clean_data["Temperature"]:
        try:
            item = float(item)
        except ValueError:
            drop_index_2 = clean_data[clean_data["Temperature"] == item].index
            clean_data = clean_data.drop(drop_index_2, axis = 0)



# Filter out null value
not_null_condition = clean_data["Antenna1 GetTxPower"].notnull()
clean_data = clean_data[not_null_condition]

# Reset index column
clean_data = clean_data.reset_index(drop = True)

# Add Date_time column and set as index column
clean_data["Date_Time"] = clean_data["TestDate"] + " " + clean_data["Time"]
clean_data = clean_data.set_index(["Date_Time"])

<ipython-input-6-17e32c1f5f1c>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data["Antenna1 GetTxPower"] = clean_data["Antenna1 GetTxPower"].astype("float")


In [7]:
[1, 2, 3] + [10, 11, 12]

[1, 2, 3, 10, 11, 12]